![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Crafting and Optimizing Context

## From RAG Basics to Practical Context Engineering

In the previous notebook, you built a working RAG system and saw why context quality matters. Now you'll learn to engineer context with professional-level rigor.

**What makes context "good"?**

This notebook teaches you that **context engineering is real engineering** - it requires the same rigor, analysis, and deliberate decision-making as any other engineering discipline. Context isn't just "data you feed to an LLM" - it requires thoughtful preparation, quality assessment, and optimization.

## What You'll Learn

**The Engineering Mindset:**
- Why context quality matters (concrete impact on accuracy, relevance, cost)
- The transformation workflow: Raw Data → Engineered Context → Quality Responses
- Contrasts between naive and engineered approaches

**Data Engineering for Context:**
- Systematic transformation: Extract → Clean → Transform → Optimize → Store
- Engineering decisions based on YOUR domain requirements
- When to use different approaches (RAG, Structured Views, Hybrid)

**Introduction to Chunking:**
- When does your data need chunking? (Critical first question)
- Different chunking strategies and their trade-offs
- How to choose based on YOUR data characteristics

**Context Preparation Pipelines:**
- Three pipeline architectures (Request-Time, Batch, Event-Driven)
- How to choose based on YOUR constraints
- Building reusable context preparation workflows

**Time to complete:** 90-105 minutes

---

## Prerequisites

- Completed Section 2, Notebook 1 (RAG Fundamentals and Implementation)
- Redis 8 running locally
- OpenAI API key set
- Understanding of RAG basics and vector embeddings

---

## Part 1: Context is Data - and Data Requires Engineering

### The Naive Approach (What NOT to Do)

Let's start by seeing what happens when you treat context as "just data" without engineering discipline.

**Scenario:** A student asks "What machine learning courses are available?"

Let's see what happens with a naive approach:

### Setup

In [1]:
import os
import sys

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify required environment variables
required_vars = ["OPENAI_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(
        f"""⚠️  Missing required environment variables: {', '.join(missing_vars)}

Please create a .env file with:
OPENAI_API_KEY=your_openai_api_key
REDIS_URL=redis://localhost:6379
"""
    )
    sys.exit(1)

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print("✅ Environment variables loaded")

✅ Environment variables loaded


In [2]:
import asyncio

# Import dependencies
import json
from typing import Any, Dict, List

import redis
import tiktoken
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from redis_context_course import CourseManager, redis_config

# Initialize
course_manager = CourseManager()
redis_client = redis.from_url(REDIS_URL, decode_responses=True)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Token counter
encoding = tiktoken.encoding_for_model("gpt-4o")


def count_tokens(text: str) -> int:
    return len(encoding.encode(text))


print("✅ Dependencies loaded")

08:43:19 redisvl.index.index INFO   Index already exists, not overwriting.


✅ Dependencies loaded

### Naive Approach: Dump Everything

The simplest approach is to include all course data in every request:

In [3]:
# Naive Approach: Get all courses and dump as JSON
all_courses = await course_manager.get_all_courses()

# Convert to raw JSON (what many developers do first)
raw_context = json.dumps(
    [
        {
            "id": c.id,
            "course_code": c.course_code,
            "title": c.title,
            "description": c.description,
            "department": c.department,
            "credits": c.credits,
            "difficulty_level": c.difficulty_level.value,
            "format": c.format.value,
            "instructor": c.instructor,
            "prerequisites": (
                [p.course_code for p in c.prerequisites] if c.prerequisites else []
            ),
            "created_at": str(c.created_at) if hasattr(c, "created_at") else None,
            "updated_at": str(c.updated_at) if hasattr(c, "updated_at") else None,
        }
        for c in all_courses[:10]  # Just first 10 for demo
    ],
    indent=2,
)

token_count = count_tokens(raw_context)

print(
    f"""📊 Naive Approach Results:
   Courses included: {len(all_courses[:10])}
   Token count: {token_count:,}
   Estimated cost per request: ${(token_count / 1_000_000) * 2.50:.4f}

   For 100 courses, this would be ~{token_count * 10:,} tokens!
"""
)

# Show a sample
print("\n📄 Sample of raw JSON context:")
print(raw_context[:500] + "...")

08:43:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


📊 Naive Approach Results:
   Courses included: 10
   Token count: 1,689
   Estimated cost per request: $0.0042

   For 100 courses, this would be ~16,890 tokens!


📄 Sample of raw JSON context:
[
  {
    "id": "course_catalog:01K9A41NZ4FAYCBY18A6Z1Y86H",
    "course_code": "CS004",
    "title": "Database Systems",
    "description": "Design and implementation of database systems. SQL, normalization, transactions, and database administration.",
    "department": "Computer Science",
    "credits": 3,
    "difficulty_level": "intermediate",
    "format": "online",
    "instructor": "John Zamora",
    "prerequisites": [],
    "created_at": "2025-11-05 08:43:20.429564",
    "updated_at": "2...


In [4]:
[course.title for course in all_courses]

['Database Systems',
 'Web Development',
 'Web Development',
 'Web Development',
 'Web Development',
 'Linear Algebra',
 'Linear Algebra',
 'Linear Algebra',
 'Linear Algebra',
 'Linear Algebra',
 'Calculus I',
 'Calculus I',
 'Calculus I',
 'Calculus I',
 'Calculus I',
 'Marketing Strategy',
 'Marketing Strategy',
 'Marketing Strategy',
 'Marketing Strategy',
 'Marketing Strategy',
 'Marketing Strategy',
 'Marketing Strategy',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Cognitive Psychology',
 'Data Structures and Algorithms',
 'Principles of Management',
 'Principles of Management',
 'Principles of Management',
 'Introduction to Psychology',
 'Introduction to Psychology',
 'Introduction to Psychology',
 'Data Visualization',
 'Data Visualization',
 'Data Visualization',
 'Data Visualization',
 'Machine Learning',
 'Introduction to Programming',
 'Introduction to Programming',
 

### Test the Naive Approach

In [5]:
# Test with a real query
query = "What machine learning courses are available?"

messages = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{raw_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]

response = llm.invoke(messages)

print(
    f"""🤖 Query: "{query}"

Response:
{response.content}
"""
)

08:43:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🤖 Query: "What machine learning courses are available?"

Response:
Currently, there are no machine learning courses listed in the available course catalog. If you are interested in machine learning, you might consider exploring related courses such as "Database Systems" or "Linear Algebra," which can provide foundational knowledge useful in the field of machine learning.



### Problems with the Naive Approach

As discussed in previous notebooks, this approach has several problems:

1. **Excessive Token Usage**
   - 10 courses = ~1,703 tokens
   - 100 courses would be ~17,030 tokens


2. **Raw JSON is Inefficient**
   - Includes internal fields (IDs, timestamps, created_at, updated_at)
   - Verbose formatting (indentation, field names repeated)


3. **No Filtering**
   - Student asked about ML, but got all courses, even irrelevant ones
   - **Dilutes relevant information with noise**


4. **Poor Response Quality**
   - Generic responses ("We have many courses...")
   - May miss the most relevant courses
   - Can't provide personalized recommendations


5. **Not Scalable**
   - What if you have 1,000 courses? 10,000?
   - What if courses change daily?
   - Requires code changes to update

**Therefore, the goal is not only to give the LLM "all the data" - it's to *give it the useful data.***

---

## The Engineering Mindset

Context is data that flows through a pipeline. Like any data engineering problem, it requires:

### 1. Requirements Analysis
- What is the intended use case?
- What queries will users ask?
- What information do they need?
- What constraints exist (token budget, latency, cost)?

### 2. Data Transformation
- Raw data → Cleaned data → Structured data → LLM-optimized context

### 3. Quality Metrics
- How do we measure if context is "good"?
- Relevance, completeness, efficiency, accuracy

### 4. Testing and Iteration
- Test with real queries
- Measure quality metrics
- Iterate based on results

**The Engineering Question:** "How do we transform raw course data into high-quality context that produces accurate, relevant, efficient responses?"

### Three Engineering Approaches

Let's compare three approaches with concrete examples:

| Approach | Description | Token Usage | Response Quality | Maintenance | Verdict |
|----------|-------------|-------------|------------------|-------------|---------|
| **Naive** | Include all raw data | 50K tokens | Poor (generic) | Easy | ❌ Not practical |
| **RAG** | Semantic search for relevant courses | 3K tokens | Good (relevant) | Moderate | ✅ Good for most cases |
| **Structured Views** | Pre-compute LLM-optimized summaries | 2K tokens | Excellent (overview + details) | Higher | ✅ Best for real-world use |
| **Hybrid** | Structured view + RAG | 5K tokens | Excellent (best of both) | Higher | ✅ Best for real-world use |

Let's implement each approach and compare them.

---

## Part 2: Data Engineering Workflow - From Raw to Optimized

### The Data Engineering Pipeline

Context preparation follows a systematic workflow:

```
Raw Data (Database/API)
    ↓
[Step 1: Extract] - Get the data
    ↓
[Step 2: Clean] - Remove noise, fix inconsistencies
    ↓
[Step 3: Transform] - Structure for LLM consumption
    ↓
[Step 4: Optimize] - Reduce tokens, improve clarity
    ↓
[Step 5: Store] - Vector DB, cache, or pre-compute
    ↓
Engineered Context (Ready for LLM)
```

Let's walk through this pipeline with a real example.

### Step 1: Extract (Raw Data)

First, let's look at what raw course data looks like:

In [6]:
# Get a sample course
sample_course = all_courses[0]

# Show raw database record
raw_record = {
    "id": sample_course.id,
    "course_code": sample_course.course_code,
    "title": sample_course.title,
    "description": sample_course.description,
    "department": sample_course.department,
    "credits": sample_course.credits,
    "difficulty_level": sample_course.difficulty_level.value,
    "format": sample_course.format.value,
    "instructor": sample_course.instructor,
    "prerequisites": (
        [p.course_code for p in sample_course.prerequisites]
        if sample_course.prerequisites
        else []
    ),
    "created_at": (
        str(sample_course.created_at)
        if hasattr(sample_course, "created_at")
        else "2024-01-15T08:30:00Z"
    ),
    "updated_at": (
        str(sample_course.updated_at)
        if hasattr(sample_course, "updated_at")
        else "2024-09-01T14:22:00Z"
    ),
}

raw_json = json.dumps(raw_record, indent=2)
raw_tokens = count_tokens(raw_json)

print("📄 Step 1: Raw Database Record")
print("=" * 80)
print(raw_json)
print("=" * 80)
print(f"\n📊 Token count: {raw_tokens}")

📄 Step 1: Raw Database Record
{
  "id": "course_catalog:01K9A41NZ4FAYCBY18A6Z1Y86H",
  "course_code": "CS004",
  "title": "Database Systems",
  "description": "Design and implementation of database systems. SQL, normalization, transactions, and database administration.",
  "department": "Computer Science",
  "credits": 3,
  "difficulty_level": "intermediate",
  "format": "online",
  "instructor": "John Zamora",
  "prerequisites": [],
  "created_at": "2025-11-05 08:43:20.429564",
  "updated_at": "2025-11-05 08:43:20.429571"
}

📊 Token count: 161


Issues with above:
 - Internal fields (IDs, timestamps) waste tokens
 - Verbose JSON formatting
 - Prerequisites are codes, not human-readable
 - No structure for LLM consumption

### Step 2: Clean (Remove Noise)

Remove fields that don't help the LLM answer user queries:

In [7]:
# Step 2: Clean - Remove internal fields
cleaned_record = {
    "course_code": sample_course.course_code,
    "title": sample_course.title,
    "description": sample_course.description,
    "department": sample_course.department,
    "credits": sample_course.credits,
    "difficulty_level": sample_course.difficulty_level.value,
    "format": sample_course.format.value,
    "instructor": sample_course.instructor,
    "prerequisites": (
        [p.course_code for p in sample_course.prerequisites]
        if sample_course.prerequisites
        else []
    ),
}

cleaned_json = json.dumps(cleaned_record, indent=2)
cleaned_tokens = count_tokens(cleaned_json)

print("📄 Step 2: Cleaned Record")
print("=" * 80)
print(cleaned_json)
print("=" * 80)
print(
    f"\n📊 Token count: {cleaned_tokens} (saved {raw_tokens - cleaned_tokens} tokens, {((raw_tokens - cleaned_tokens) / raw_tokens * 100):.1f}% reduction)"
)

📄 Step 2: Cleaned Record
{
  "course_code": "CS004",
  "title": "Database Systems",
  "description": "Design and implementation of database systems. SQL, normalization, transactions, and database administration.",
  "department": "Computer Science",
  "credits": 3,
  "difficulty_level": "intermediate",
  "format": "online",
  "instructor": "John Zamora",
  "prerequisites": []
}

📊 Token count: 89 (saved 72 tokens, 44.7% reduction)



Improvements:
 - Removed id, created_at, updated_at
 - Still has all information needed to answer queries

Still has minor problems:
 - JSON formatting is verbose (this is a *minor* issue as LLMs can handle it; however)
 - Prerequisites are still codes


### Step 3: Transform (Structure for LLM)

Convert to a format optimized for LLM consumption:

In [8]:
# Step 3: Transform - Convert to LLM-friendly format


def transform_course_to_text(course) -> str:
    """Transform course object to LLM-optimized text format."""

    # Build prerequisites text
    prereq_text = ""
    if course.prerequisites:
        prereq_codes = [p.course_code for p in course.prerequisites]
        prereq_text = f"\nPrerequisites: {', '.join(prereq_codes)}"

    # Build course text
    course_text = f"""{course.course_code}: {course.title}
Department: {course.department}\nCredits: {course.credits}\nLevel: {course.difficulty_level.value}\nFormat: {course.format.value}
Instructor: {course.instructor}{prereq_text}
Description: {course.description}
    """

    return course_text


transformed_text = transform_course_to_text(sample_course)
transformed_tokens = count_tokens(transformed_text)

print("📄 Step 3: Transformed to LLM-Friendly Format")
print("=" * 80)
print(transformed_text)
print("=" * 80)
print(
    f"\n📊 Token count: {transformed_tokens} (saved {cleaned_tokens - transformed_tokens} tokens, {((cleaned_tokens - transformed_tokens) / cleaned_tokens * 100):.1f}% reduction)"
)

📄 Step 3: Transformed to LLM-Friendly Format
CS004: Database Systems
Department: Computer Science
Credits: 3
Level: intermediate
Format: online
Instructor: John Zamora
Description: Design and implementation of database systems. SQL, normalization, transactions, and database administration.
    

📊 Token count: 50 (saved 39 tokens, 43.8% reduction)



✅ Improvements:
 - Natural text format with the correct metadata
 - Clear structure with labels
 - No JSON overhead (brackets, quotes, commas)

**Note:** In case the description is too long, we can apply compression techniques such as summarization to keep the description within a desired token limit. Section 3 will cover compression in more detail.


### Step 4: Optimize (Further Reduce Tokens)

For even more efficiency, we can create a summarized version:

In [9]:
# Step 4: Optimize - Create ultra-compact version
# TODO: Maybe use summarization here? Maybe for that we need a longer description or some other metadata?

def optimize_course_text(course) -> str:
    """Create ultra-compact course description."""
    prereqs = (
        f" (Prereq: {', '.join([p.course_code for p in course.prerequisites])})"
        if course.prerequisites
        else ""
    )
    return (
        f"{course.course_code}: {course.title} - {course.description[:100]}...{prereqs}"
    )


optimized_text = optimize_course_text(sample_course)
optimized_tokens = count_tokens(optimized_text)

print("📄 Step 4: Optimized (Ultra-Compact)")
print("=" * 80)
print(optimized_text)
print("=" * 80)
print(
    f"\n📊 Token count: {optimized_tokens} (saved {transformed_tokens - optimized_tokens} tokens, {((transformed_tokens - optimized_tokens) / transformed_tokens * 100):.1f}% reduction)"
)

📄 Step 4: Optimized (Ultra-Compact)
CS004: Database Systems - Design and implementation of database systems. SQL, normalization, transactions, and database admini...

📊 Token count: 24 (saved 26 tokens, 52.0% reduction)


Improvements:
   - Truncated description to 100 chars
   - Removed metadata (instructor, format, credits)

Trade-off:
   - Lost some detail (may need for specific queries)
   - Best for overview/catalog views

**Note:** This is just an example of what you can do to be more efficient. This is where you have to be creative and engineer based on the usercase and requirements.

### Step 5: Store (Choose Storage Strategy)

Now we need to decide HOW to store this engineered context:

**Option 1: Vector Database (RAG)**
- Store transformed text with embeddings
- Retrieve relevant courses at query time
- Good for: Large datasets, specific queries

**Option 2: Pre-Computed Views**
- Create structured summaries ahead of time
- Store in Redis as cached views
- Good for: Common queries, overview information

**Option 3: Hybrid**
- Combine both approaches
- Pre-compute catalog view + RAG for details
- Good for: Real-world systems

Let's implement all three and compare.

### Summary: The Transformation Pipeline

Let's see the complete transformation:

In [10]:
print("=" * 80)
print("EXAMPLE PIPELINE SUMMARY")
print("=" * 80)

print(
    f"""
Step 1: Raw Database Record
   Token count: {raw_tokens}
   Format: JSON with all fields

Step 2: Cleaned Record
   Token count: {cleaned_tokens} ({((raw_tokens - cleaned_tokens) / raw_tokens * 100):.1f}% reduction)
   Removed: Internal fields (IDs, timestamps)

Step 3: Transformed to LLM Format
   Token count: {transformed_tokens} ({((cleaned_tokens - transformed_tokens) / cleaned_tokens * 100):.1f}% reduction from Step 2)
   Format: Natural text, structured

Step 4: Optimized (Ultra-Compact)
   Token count: {optimized_tokens} ({((transformed_tokens - optimized_tokens) / transformed_tokens * 100):.1f}% reduction from Step 3)
   Format: Single line, truncated

TOTAL REDUCTION: {raw_tokens} → {optimized_tokens} tokens ({((raw_tokens - optimized_tokens) / raw_tokens * 100):.1f}% reduction)
"""
)

print("=" * 80)
print("\n🎯 Key Insight:")
print("   Through systematic engineering, we reduced token usage by ~70%")
print("   while IMPROVING readability for the LLM!")
print("=" * 80)

EXAMPLE PIPELINE SUMMARY

Step 1: Raw Database Record
   Token count: 161
   Format: JSON with all fields

Step 2: Cleaned Record
   Token count: 89 (44.7% reduction)
   Removed: Internal fields (IDs, timestamps)

Step 3: Transformed to LLM Format
   Token count: 50 (43.8% reduction from Step 2)
   Format: Natural text, structured

Step 4: Optimized (Ultra-Compact)
   Token count: 24 (52.0% reduction from Step 3)
   Format: Single line, truncated

TOTAL REDUCTION: 161 → 24 tokens (85.1% reduction)


🎯 Key Insight:
   Through systematic engineering, we reduced token usage by ~70%
   while IMPROVING readability for the LLM!


The key insight states that we reduced token usage.

However, it should be noted that reduction is not the goal. The goal is to optimize the content and provide the most relevant information to the LLM.

---

## Part 3: Engineering Decision - When to Use Each Approach

Now let's implement the three approaches and compare them with real queries.

### Approach 1: RAG (Semantic Search)

Retrieve only relevant courses using vector search:

In [11]:
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery
from redisvl.query.filter import Tag

# Initialize vector search
index_name = "course_index"

# Check if index exists, create if not
try:
    index = SearchIndex.from_existing(index_name, redis_url=REDIS_URL)
    print(f"✅ Using existing index: {index_name}")
except:
    print(
        f"⚠️  Index '{index_name}' not found. Please run Section 2 notebooks to create it."
    )
    print("   For this demo, we'll simulate RAG results.")
    index = None

⚠️  Index 'course_index' not found. Please run Section 2 notebooks to create it.
   For this demo, we'll simulate RAG results.


In [12]:
# Simulate RAG retrieval (in production, this would use vector search)


async def rag_approach(query: str, limit: int = 5) -> str:
    """Retrieve relevant courses using semantic search."""

    # In production: Use vector search
    # For demo: Filter courses by keyword matching
    query_lower = query.lower()

    relevant_courses = []
    for course in all_courses:
        # Simple keyword matching (in production, use embeddings)
        if any(
            keyword in course.title.lower() or keyword in course.description.lower()
            for keyword in ["machine learning", "ml", "ai", "data science", "neural"]
        ):
            relevant_courses.append(course)
            if len(relevant_courses) >= limit:
                break

    # Transform to LLM-friendly format
    context = "\n\n".join([transform_course_to_text(c) for c in relevant_courses])
    return context


# Test RAG approach
query = "What machine learning courses are available?"
rag_context = await rag_approach(query, limit=5)
rag_tokens = count_tokens(rag_context)

print(
    f"""📊 RAG Approach Results:
   Query: "{query}"
   Courses retrieved: 5
   Token count: {rag_tokens:,}

📄 Context Preview:
{rag_context[:500]}...
"""
)

📊 RAG Approach Results:
   Query: "What machine learning courses are available?"
   Courses retrieved: 5
   Token count: 270

📄 Context Preview:
CS003: Web Development
Department: Computer Science
Credits: 3
Level: intermediate
Format: in_person
Instructor: Cody Ayala
Description: Full-stack web development using modern frameworks. HTML, CSS, JavaScript, React, and backend APIs.
    

CS002: Web Development
Department: Computer Science
Credits: 3
Level: intermediate
Format: in_person
Instructor: Kimberly Robertson
Description: Full-stack web development using modern frameworks. HTML, CSS, JavaScript, React, and backend APIs.
    

CS008:...



### Approach 2: Structured Views (Pre-Computed Summaries)

Create a pre-computed catalog view that's optimized for LLM consumption:

In [13]:
# Approach 2: Structured Views
# Pre-compute a catalog summary organized by department


async def create_catalog_view() -> str:
    """Create a pre-computed catalog view organized by department."""

    # Group courses by department
    by_department = {}
    for course in all_courses:
        dept = course.department
        if dept not in by_department:
            by_department[dept] = []
        by_department[dept].append(course)

    # Build catalog view
    catalog_sections = []

    for dept_name in sorted(by_department.keys()):
        courses = by_department[dept_name]

        # Create department section
        dept_section = f"\n## {dept_name} ({len(courses)} courses)\n"

        # Add course summaries (optimized format)
        course_summaries = []
        for course in courses[:10]:  # Limit for demo
            summary = f"- {course.course_code}: {course.title} ({course.difficulty_level.value})"
            course_summaries.append(summary)

        dept_section += "\n".join(course_summaries)
        catalog_sections.append(dept_section)

    catalog_view = "# Redis University Course Catalog\n" + "\n".join(catalog_sections)
    return catalog_view


# Create and cache the view
catalog_view = await create_catalog_view()
catalog_tokens = count_tokens(catalog_view)

# Store in Redis for reuse
redis_client.set("course_catalog_view", catalog_view)

print(
    f"""📊 Structured View Approach Results:
   Total courses: {len(all_courses)}
   Token count: {catalog_tokens:,}
   Cached in Redis: ✅

📄 Catalog Preview:
{catalog_view[:600]}...
"""
)

📊 Structured View Approach Results:
   Total courses: 50
   Token count: 585
   Cached in Redis: ✅

📄 Catalog Preview:
# Redis University Course Catalog

## Business (10 courses)
- BUS033: Marketing Strategy (intermediate)
- BUS035: Marketing Strategy (intermediate)
- BUS032: Marketing Strategy (intermediate)
- BUS034: Marketing Strategy (intermediate)
- BUS037: Marketing Strategy (intermediate)
- BUS039: Marketing Strategy (intermediate)
- BUS040: Marketing Strategy (intermediate)
- BUS038: Principles of Management (beginner)
- BUS036: Principles of Management (beginner)
- BUS031: Principles of Management (beginner)

## Computer Science (10 courses)
- CS004: Database Systems (intermediate)
- CS003: Web Develo...



### Approach 3: Hybrid (Best of Both Worlds)

Combine structured view (overview) + RAG (specific details):

In [14]:
# Approach 3: Hybrid


async def hybrid_approach(query: str) -> str:
    """Combine catalog overview with RAG for specific details."""

    # Part 1: Get catalog overview (from cache)
    catalog_overview = redis_client.get("course_catalog_view")

    # Part 2: Get specific course details via RAG
    specific_courses = await rag_approach(query, limit=3)

    # Combine
    hybrid_context = f"""# Course Catalog Overview
{catalog_overview}

---

# Detailed Information for Your Query
{specific_courses}
"""

    return hybrid_context


# Test hybrid approach
hybrid_context = await hybrid_approach(query)
hybrid_tokens = count_tokens(hybrid_context)

print(
    f"""📊 Hybrid Approach Results:
   Query: "{query}"
   Token count: {hybrid_tokens:,}

   Components:
   - Catalog overview: {catalog_tokens:,} tokens
   - Specific details (RAG): {rag_tokens:,} tokens

📄 Context Structure:
   1. Full catalog overview (all departments)
   2. Detailed info for 3 most relevant courses
"""
)

📊 Hybrid Approach Results:
   Query: "What machine learning courses are available?"
   Token count: 760

   Components:
   - Catalog overview: 585 tokens
   - Specific details (RAG): 270 tokens

📄 Context Structure:
   1. Full catalog overview (all departments)
   2. Detailed info for 3 most relevant courses



### Compare All Three Approaches

Let's test all three with the same query and compare results:

In [15]:
# Test all three approaches
query = "What machine learning courses are available?"

print("=" * 80)
print("COMPARING THREE APPROACHES")
print("=" * 80)

# Approach 1: RAG
messages_rag = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

Available Courses:
{rag_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_rag = llm.invoke(messages_rag)

# Approach 2: Structured View
messages_view = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{catalog_view}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_view = llm.invoke(messages_view)

# Approach 3: Hybrid
messages_hybrid = [
    SystemMessage(
        content=f"""You are a Redis University course advisor.

{hybrid_context}

Help students find relevant courses."""
    ),
    HumanMessage(content=query),
]
response_hybrid = llm.invoke(messages_hybrid)

# Display comparison
print(
    f"""
Query: "{query}"

{'=' * 80}
APPROACH 1: RAG (Semantic Search)
{'=' * 80}
Token count: {rag_tokens:,}
Response:
{response_rag.content}

{'=' * 80}
APPROACH 2: Structured View (Pre-Computed)
{'=' * 80}
Token count: {catalog_tokens:,}
Response:
{response_view.content}

{'=' * 80}
APPROACH 3: Hybrid (View + RAG)
{'=' * 80}
Token count: {hybrid_tokens:,}
Response:
{response_hybrid.content}

{'=' * 80}
"""
)

COMPARING THREE APPROACHES


08:43:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


08:43:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


08:43:26 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: "What machine learning courses are available?"

APPROACH 1: RAG (Semantic Search)
Token count: 270
Response:
I'm sorry, but there are no machine learning courses currently available in the course list provided. If you are interested in related fields, you might consider taking "MATH022: Linear Algebra," as it covers essential topics like vector spaces and matrices that are foundational for machine learning.

APPROACH 2: Structured View (Pre-Computed)
Token count: 585
Response:
We offer one advanced machine learning course:

- CS007: Machine Learning (advanced)

APPROACH 3: Hybrid (View + RAG)
Token count: 760
Response:
The available machine learning course is:

- CS007: Machine Learning (advanced)




### Decision Framework: Which Approach to Use?

Here's how to choose based on YOUR requirements:

| Factor | RAG | Structured Views | Hybrid |
|--------|-----|------------------|--------|
| **Token Efficiency** | ✅ Good (3K) | ✅✅ Excellent (2K) | ⚠️ Moderate (5K) |
| **Response Quality** | ✅ Good (relevant) | ✅ Good (overview) | ✅✅ Excellent (both) |
| **Latency** | ⚠️ Moderate (search) | ✅✅ Fast (cached) | ⚠️ Moderate (search) |
| **Maintenance** | ✅ Low (auto-updates) | ⚠️ Higher (rebuild views) | ⚠️ Higher (both) |
| **Best For** | Specific queries | Overview queries | Real-world systems |

**Decision Process:**

1. **Analyze YOUR data characteristics:**
   - How many items? (10s, 100s, 1000s, millions?)
   - How often does it change? (Real-time, daily, weekly?)
   - What's the average item size? (100 words, 1000 words, 10K words?)

2. **Analyze YOUR query patterns:**
   - Specific queries ("Show me RU101") → RAG
   - Overview queries ("What courses exist?") → Structured Views
   - Mixed queries → Hybrid

3. **Analyze YOUR constraints:**
   - Tight token budget → Structured Views
   - Real-time updates required → RAG
   - Best quality needed → Hybrid

**Example Decision:**

For Redis University:
- ✅ **Data:** 100-500 courses, updated weekly, 200-500 words each
- ✅ **Queries:** Mix of overview ("What's available?") and specific ("ML courses?")
- ✅ **Constraints:** Moderate token budget, weekly updates acceptable
- ✅ **Decision:** **Hybrid approach** (pre-compute catalog + RAG for details)

---

## Part 4: Introduction to Chunking - When and Why

So far, we've worked with course data where each course is a complete, self-contained unit. But what happens when you have **long documents** that contain multiple distinct topics?

This is where **chunking** *may* become necessary—but it's not always the right choice.

### The Critical First Question: Does My Data Need Chunking?

**Chunking is NOT a default step** - it's an engineering decision that depends on multiple factors:

1. **Document type:** Structured records vs. long-form text vs. PDFs
2. **Data characteristics:** Small discrete items vs. large continuous documents
3. **Application requirements:** Query patterns, retrieval precision needs
4. **Embedding model limitations:** Context window size

⚠️ **Important:** With modern long-context models (128K+ tokens), "fitting in context" is less of a concern. The real question is about **retrieval precision** and **data modeling**.

Let's understand when chunking helps—and when it hurts.

### The "Don't Chunk" Strategy: A Valid Option

For structured data and naturally small documents, **not chunking is often the best strategy**:

**Examples where whole-record embedding works better:**
- ✅ **Course catalogs** - Each course is a complete, self-contained unit
- ✅ **Product listings** - All product info should be retrieved together
- ✅ **FAQ entries** - Question + answer form an atomic unit
- ✅ **Database records** - Structured data with natural boundaries
- ✅ **Support tickets** - Single issue with context

**Why "don't chunk" works here:**
- Each unit is **semantically complete** - all relevant info is together
- Natural boundaries exist - chunking would split related information
- Retrieval precision is maximized - you get exactly what's relevant
- Simpler implementation - no chunking logic, no overlap decisions

> 💡 **Key Insight:** For our course catalog, each course is already an optimal retrieval unit. Chunking it would only hurt quality.

In [16]:
# Example: Course data (NO chunking needed)
sample_course_text = transform_course_to_text(all_courses[0])
sample_tokens = count_tokens(sample_course_text)

print(
    f"""📊 Example: Course Description
{'=' * 80}
{sample_course_text}
{'=' * 80}

Token count: {sample_tokens}
Semantic completeness: ✅ Complete (has all info about this course)
Chunking needed? ❌ NO

Why whole-record embedding works here:
- Self-contained (doesn't reference other sections)
- Semantically complete (has all course details)
- Natural boundary (one course = one retrieval unit)
- Breaking it up would hurt retrieval quality
"""
)

📊 Example: Course Description
CS004: Database Systems
Department: Computer Science
Credits: 3
Level: intermediate
Format: online
Instructor: John Zamora
Description: Design and implementation of database systems. SQL, normalization, transactions, and database administration.
    

Token count: 50
Semantic completeness: ✅ Complete (has all info about this course)
Chunking needed? ❌ NO

Why not?
- Under 500 tokens (well within limits)
- Self-contained (doesn't reference other sections)
- Semantically complete (has all course details)
- Breaking it up would lose context



### When Chunking May Help

Chunking *can* improve retrieval when documents have certain characteristics:

**Document types that may benefit from chunking:**
- Research papers with multiple distinct sections
- Technical documentation spanning many topics
- Books and long-form content
- Legal contracts with multiple clauses
- Medical records with multiple visits/conditions

**Why chunking helps in these cases:**
- **Multiple distinct topics** - Different sections should be retrieved separately for precision
- **Improves retrieval precision** - Find specific sections, not whole document
- **Better data modeling** - Just like database design, structure affects quality
- **Addresses research-documented context quality problems** (see below)

⚠️ **Note:** Modern embedding models (jina-embeddings-v2, etc.) support 8K+ tokens. Model limits are less of a concern than **retrieval precision**.

### Research Background: Why Long Context Can Hurt

Even with large context windows (128K+ tokens), research shows that **how you structure context matters more than fitting everything in**:

**1. "Lost in the Middle" (Stanford/UC Berkeley, 2023)**

*Source: [arXiv:2307.03172](https://arxiv.org/abs/2307.03172), published in TACL 2024*

LLMs exhibit a **"U-shaped" attention pattern**:
- High recall for information at the **beginning** and **end** of context
- **Significantly degraded performance** for information in the middle
- This happens even in models explicitly designed for long contexts

**Implication for chunking:** For long documents, chunking ensures the relevant section is retrieved and placed prominently in context, rather than buried in the middle of a 50-page document.

**2. "Context Rot" (Chroma Research, 2025)**

*Source: [research.trychroma.com/context-rot](https://research.trychroma.com/context-rot)*

Tested 18 LLMs (GPT-4.1, Claude 4, Gemini 2.5, Qwen3) and found:
- **Performance degrades as input length increases**, even when relevant info is present
- **Distractor effect**: Irrelevant content actively hurts model performance
- Even 4 distractor documents can significantly degrade output quality
- Position of irrelevant content matters (middle is worst)

**Implication for chunking:** Retrieving smaller, focused chunks reduces "distractor tokens" that poison the context.

**3. Needle in the Haystack (NIAH) Benchmark**

*Source: [Greg Kamradt, github.com/gkamradt/LLMTest_NeedleInAHaystack](https://github.com/gkamradt/LLMTest_NeedleInAHaystack)*

Tests whether LLMs can find a specific fact ("needle") buried in long context ("haystack"):
- Models often **fail to retrieve information** even when it's present in context
- Performance varies by position (middle is typically worst)
- **Limitation**: NIAH tests lexical retrieval only, not semantic understanding

**Implication for chunking:** For structured data like course catalogs, NIAH is **irrelevant**—each record IS the needle. Chunking would only fragment complete units.

---

**The Key Insight:**

These research findings don't prescribe a universal chunking rule—they inform your design decisions:

- **Structured records** (courses, products, FAQs): The "lost in the middle" problem typically doesn't apply because each record is already a focused, atomic unit. However, if your records are unusually large or contain multiple distinct topics, chunking may still help.

- **Long-form documents**: Context rot and positional bias become more relevant as document length increases, but the degree depends on your specific content, query patterns, and model capabilities. Chunking can help surface relevant sections, but over-chunking fragments context.

- **Mixed content types**: Real-world data rarely fits neat categories. A research paper with embedded tables, a product listing with extensive reviews, or a FAQ with nested sub-questions all require case-by-case judgment.

The research provides **mental models for reasoning about trade-offs**, not binary rules. Experiment with your actual data and queries.

### Emerging Strategies (2024)

Recent research has introduced new approaches that challenge traditional chunking:

1. **Late Chunking** (Jina AI): Embed the entire document first, then chunk the embeddings. Preserves cross-chunk context without ColBERT-level storage costs.

2. **Contextual Retrieval** (Anthropic): Add LLM-generated context to each chunk before embedding. Can reduce retrieval failures by 49-67%.

3. **Hybrid Search**: Combine vector embeddings with BM25 keyword search. Often outperforms either approach alone.

The key insight: **chunking strategy is a design choice, not a one-size-fits-all prescription**.


---

## Advanced Chunking: Three Distinct Document Types

**Critical Insight:** Chunking is **subjective and document-dependent**. There is no universal strategy.

The optimal approach depends entirely on:
- **Document type and structure** (research papers vs. contracts vs. transcripts)
- **Content heterogeneity** (text, tables, equations, code, images)
- **Query patterns** (specific facts vs. conceptual understanding)
- **Domain requirements** (legal precision vs. scientific accuracy vs. conversational context)
- **Models being used** (embedding model capabilities, LLM context windows)
- **Performance metrics** (retrieval precision, answer quality, latency, cost)

This section explores three fundamentally different document types that require distinct chunking approaches:

1. **Research Papers & Long-Form Documents** (Multimodal Content)
2. **Legal Contracts & Clause-Level Documents** (Advanced Data Engineering)
3. **Transcripts & Meeting Notes** (Temporal & Speaker-Based Chunking)


### Document Type 1: Research Papers & Long-Form Documents (Multimodal Content)

**⚠️ Important:** This category is **broad and subjective**. It includes:
- Research papers (scientific, technical, medical)
- Books and textbooks
- Technical documentation
- Financial reports
- Medical records
- Product manuals
- **Even legal contracts** (which we'll discuss separately due to unique challenges)

**What makes these documents similar:**
- Heterogeneous content types (text, tables, equations, figures, code)
- Clear or implicit structural hierarchy (sections, chapters, subsections)
- Multiple distinct topics within single document
- Cross-references between sections
- Length varies from 10 pages to 1,000+ pages

**Key Challenge:** Multimodal content requires specialized handling to preserve meaning.

#### Why Chunking Matters: Research-Backed Benefits

**1. Token Limit Management**
- Ensures content fits within model context windows
- Even with 128K+ context models, focused chunks improve processing efficiency
- Reduces computational costs and latency
- *Source: Anthropic Contextual Retrieval (2024)*

**2. Improved Retrieval Precision**
- Smaller, focused chunks create more accurate embeddings
- **49-67% reduction in retrieval failures** with proper chunking + contextual retrieval
- Reduces "distractor effect" from irrelevant content
- *Source: Anthropic Contextual Retrieval, September 2024*

**3. Context Preservation**
- Keeps semantically related information together
- Maintains author's logical flow and argumentation
- Prevents fragmentation of complex ideas
- **Chunk overlap (10-20%)** helps preserve context across boundaries

**4. Computational Efficiency**
- Faster processing with smaller chunks
- Reduced costs (fewer tokens per query)
- Better cache hit rates in production systems
- *Source: "Advancing Semantic Caching for LLMs" (Redis/Virginia Tech, arXiv:2504.02268, 2025)*

#### Chunking Strategies for Long-Form Documents

**Strategy 1: Page-Level Chunking**
- **Best for:** Financial reports, legal documents, books with natural page boundaries
- **How:** Split by page numbers, preserving page-level context
- **Pros:** Natural boundaries, easy to cite sources ("See page 42")
- **Cons:** May split mid-sentence or mid-paragraph

**Strategy 2: Structure-Aware Chunking (Markdown/HTML)**
- **Best for:** Documents with clear hierarchical structure (research papers, technical docs)
- **How:** Split by headings (H1, H2, H3), sections, subsections
- **Pros:** Respects logical boundaries, preserves semantic units
- **Cons:** Requires structured input (Markdown, HTML, LaTeX)

**Strategy 3: Recursive Chunking**
- **Best for:** Mixed content with varying structure
- **How:** Try splitting by paragraphs → sentences → words (progressive fallback)
- **Pros:** Handles edge cases gracefully, respects natural boundaries
- **Cons:** More complex implementation

**Strategy 4: Semantic Chunking**
- **Best for:** Unstructured text without clear boundaries
- **How:** Use embeddings to detect topic shifts, group semantically similar sentences
- **Pros:** Content-aware, doesn't rely on formatting
- **Cons:** Computationally expensive, may not align with author's structure

**Strategy 5: Hybrid Approaches**
- **Best for:** Production systems with diverse document types
- **How:** Combine multiple strategies (e.g., structure-aware + semantic)
- **Pros:** Flexible, adapts to different content types
- **Cons:** More complex to implement and maintain

**⚠️ Important:** The "best" strategy depends on YOUR specific documents and query patterns. **Experimentation is required.**

#### Chunk Overlap: Preserving Context Across Boundaries

**Why overlap matters:**
- Prevents information loss at chunk boundaries
- Maintains context for sentences that span chunks
- Improves retrieval recall (same info appears in multiple chunks)

**Recommended overlap:**
- **10-20% of chunk size** is typical
- Example: 1,000-token chunks with 100-200 token overlap
- Too little: Risk losing context at boundaries
- Too much: Redundancy, increased storage/compute costs

**Example:**
```
Chunk 1: [Tokens 1-1000]
Chunk 2: [Tokens 900-1900]  ← 100 tokens overlap with Chunk 1
Chunk 3: [Tokens 1800-2800] ← 100 tokens overlap with Chunk 2
```


#### Real-World Example: Analyzing a Research Paper

Let's analyze a **real research paper** to understand chunking decisions:

**Paper:** "Advancing Semantic Caching for LLMs with Domain-Specific Embeddings and Synthetic Data"
**Source:** arXiv:2504.02268 (Redis/Virginia Tech, April 2025)
**Length:** 12 pages, ~42,000 characters

**Paper Structure:**
- Abstract (1 paragraph, ~200 words)
- Introduction (3 pages, ~1,500 words)
- Methodology (4 pages, includes equations and algorithms)
- Experimental Results (3 pages, includes tables and figures)
- Discussion & Conclusion (2 pages)
- References (1 page)

**Content Types Present:**
1. **Text sections**: Standard prose (Introduction, Discussion)
2. **Mathematical formulas**: Contrastive loss functions, similarity metrics
3. **Tables**: Performance comparison across 13 embedding models (5 metrics each)
4. **Figures**: Bar charts showing precision/recall/F1 scores
5. **Code snippets**: Python examples for fine-tuning ModernBERT
6. **References**: Citations to related work

**Critical Question: Is chunking the ONLY way to process this paper?**

❌ **NO** - Chunking is ONE approach, but not the only one. Consider these alternatives:

**Option 1: Hierarchical Retrieval (No Chunking)**
- Create summary embedding for entire paper
- Store full paper as retrievable unit
- Use summary for initial search, full paper for detailed analysis
- **Best for:** Overview queries ("What papers discuss semantic caching?")
- **Pros:** Simple, preserves full context
- **Cons:** Poor precision for specific queries

**Option 2: Section-Based Chunking (Structure-Aware)**
- Chunk by major sections (Abstract, Intro, Methods, Results, Discussion)
- Keep tables/figures WITH their explanatory text
- Preserve mathematical formulas with variable definitions
- **Best for:** Specific queries ("What methodology did they use?")
- **Pros:** Good precision, respects paper structure
- **Cons:** Sections may still be large (1,500+ words)

**Option 3: Hybrid Approach (Recommended)**
- Summary embedding + section embeddings + full paper storage
- Multi-level retrieval: summary → relevant sections → full context
- **Best for:** Production systems with diverse query patterns
- **Pros:** Flexible, handles both overview and specific queries
- **Cons:** More complex implementation

**Option 4: Multimodal RAG**
- Extract tables/figures as separate entities
- Use vision models (GPT-4V, Claude 3) for figure understanding
- Link text chunks to related visual elements
- **Best for:** Papers with critical visual information (charts, diagrams)
- **Pros:** Handles visual content that text embeddings miss
- **Cons:** Expensive (vision model API costs), complex pipeline

**Our Expert Recommendation for This Paper:**

Use **Option 3 (Hybrid)** because:
1. Paper has clear section structure (good for structure-aware chunking)
2. Contains multimodal content (tables, figures) that need special handling
3. Queries could be overview ("What's this paper about?") or specific ("What were the precision results?")
4. Hybrid approach provides flexibility without excessive complexity


#### Handling Multimodal Content: Tables, Figures, Equations, Code

**Critical Challenge:** Research papers and technical documents contain heterogeneous content types that require specialized handling.

**Content Type 1: Tables**

**Problem:** Tables are structured data that lose meaning when chunked mid-table or separated from context.

**Best Practices:**
- ✅ **Chunk table WITH its caption and explanation**
  - Include: Table title, column headers, data rows, AND explanatory text
  - Example: "Table 1 shows HNSW performance trade-offs. As M increases from 16 to 64, recall improves from 0.89 to 0.97, but latency increases from 2.1ms to 8.7ms..."
- ✅ **Preserve table structure** in text format (Markdown, CSV, or structured description)
- ✅ **Add metadata** to chunk: table number, section, key findings
- ❌ **Don't chunk table separately** from its context - it becomes meaningless
- ❌ **Don't split tables** across multiple chunks

**Advanced Approach:** Extract tables as structured data (JSON/CSV) and store separately with links to text chunks.

**Content Type 2: Mathematical Formulas**

**Problem:** Formulas without context are meaningless. Variables need definitions.

**Best Practices:**
- ✅ **Chunk formula WITH its explanation and variable definitions**
  - Include: Formula, variable definitions, interpretation, example
  - Example: "The recall-latency trade-off can be modeled as: Latency(M, ef) = α·M² + β·ef + γ, where M = number of connections per layer, ef = size of dynamic candidate list..."
- ✅ **Keep formula + derivation + application** together
- ✅ **Include units and constraints** (e.g., "where M ∈ [16, 64]")
- ❌ **Don't separate formula** from its meaning
- ❌ **Don't chunk mid-derivation**

**Advanced Approach:** Use LaTeX-aware chunking to preserve mathematical notation.

**Content Type 3: Figures and Charts**

**Problem:** Figures contain visual information that text embeddings cannot capture.

**Best Practices:**
- ✅ **Chunk figure caption WITH its discussion in text**
  - Include: Figure number, caption, key findings, interpretation
  - Example: "Figure 1 shows precision/recall comparison across 13 embedding models. LangCache-Embed achieves 0.84 precision, outperforming OpenAI's text-embedding-3-small (0.65) by 29%..."
- ✅ **Describe visual patterns** in text (trends, comparisons, outliers)
- ✅ **Extract data from charts** if possible (convert to table/text)
- ❌ **Don't rely on figure alone** - text embeddings can't "see" images

**Advanced Approach (Multimodal RAG):**
- Use vision models (GPT-4V, Claude 3, Gemini) to generate figure descriptions
- Store figure embeddings separately using CLIP or similar vision-language models
- Link text chunks to related figures for retrieval

**Content Type 4: Code Snippets**

**Problem:** Code without context is hard to understand. Why these values? What's the use case?

**Best Practices:**
- ✅ **Chunk code WITH its context and rationale**
  - Include: Code, explanation, use case, parameter justification
  - Example: "For real-world deployment, we recommend M=32 and ef_construction=200 because this balances recall (0.94) and latency (4.3ms)..."
- ✅ **Include comments and docstrings** in the chunk
- ✅ **Add example usage** if relevant
- ❌ **Don't chunk code** without explaining WHY these values
- ❌ **Don't separate code** from its output/results

**Advanced Approach:** Use code-specific embeddings (CodeBERT, GraphCodeBERT) for better semantic understanding.

**Summary: Multimodal Content Chunking Principles**

1. **Context is king**: Never chunk content (table/formula/figure/code) without its explanation
2. **Preserve structure**: Maintain formatting that aids understanding (table structure, code indentation)
3. **Add metadata**: Enrich chunks with type, section, number (e.g., "Table 3 from Section 4.2")
4. **Consider alternatives**: For critical visual content, multimodal RAG may be necessary
5. **Experiment**: Test retrieval quality with your actual queries and documents


In [ ]:
# Example: Research paper (NEEDS chunking)
# Let's simulate a long research paper about Redis

research_paper = """
# Optimizing Vector Search Performance in Redis

## Abstract
This paper presents a comprehensive analysis of vector search optimization techniques in Redis,
examining the trade-offs between search quality, latency, and memory usage. We evaluate multiple
indexing strategies including HNSW and FLAT indexes across datasets ranging from 10K to 10M vectors.
Our results demonstrate that careful index configuration can improve search latency by up to 10x
while maintaining 95%+ recall. We also introduce novel compression techniques that reduce memory
usage by 75% with minimal impact on search quality.

## 1. Introduction
Vector databases have become essential infrastructure for modern AI applications, enabling semantic
search, recommendation systems, and retrieval-augmented generation (RAG). Redis, traditionally known
as an in-memory data structure store, has evolved to support high-performance vector search through
the RediSearch module. However, optimizing vector search performance requires understanding complex
trade-offs between multiple dimensions...

[... 5,000 more words covering methodology, experiments, results, discussion ...]

## 2. Background and Related Work
Previous work on vector search optimization has focused primarily on algorithmic improvements to
approximate nearest neighbor (ANN) search. Malkov and Yashunin (2018) introduced HNSW, which has
become the de facto standard for high-dimensional vector search. Johnson et al. (2019) developed
FAISS, demonstrating that product quantization can significantly reduce memory usage...

[... 2,000 more words ...]

## 3. Performance Analysis and Results

### 3.1 HNSW Configuration Trade-offs

Table 1 shows the performance comparison across different HNSW configurations. As M increases from 16 to 64,
we observe significant improvements in recall (0.89 to 0.97) but at the cost of increased latency (2.1ms to 8.7ms)
and memory usage (1.2GB to 3.8GB). The sweet spot for most real-world workloads is M=32 with ef_construction=200,
which achieves 0.94 recall with 4.3ms latency.

Table 1: HNSW Performance Comparison
| M  | ef_construction | Recall@10 | Latency (ms) | Memory (GB) | Build Time (min) |
|----|-----------------|-----------|--------------|-------------|------------------|
| 16 | 100            | 0.89      | 2.1          | 1.2         | 8                |
| 32 | 200            | 0.94      | 4.3          | 2.1         | 15               |
| 64 | 400            | 0.97      | 8.7          | 3.8         | 32               |

The data clearly demonstrates the fundamental trade-off between search quality and resource consumption.
For applications requiring high recall (>0.95), the increased latency and memory costs are unavoidable.

### 3.2 Mathematical Model

The recall-latency trade-off can be modeled as a quadratic function of the HNSW parameters:

Latency(M, ef) = α·M² + β·ef + γ

Where:
- M = number of connections per layer (controls graph connectivity)
- ef = size of dynamic candidate list (controls search breadth)
- α, β, γ = dataset-specific constants (fitted from experimental data)

For our e-commerce dataset, we fitted: α=0.002, β=0.015, γ=1.2 (R²=0.94)

This model allows us to predict latency for untested configurations and optimize for specific
recall targets. The quadratic dependency on M explains why doubling M more than doubles latency.

## 4. Implementation Recommendations

Based on our findings, we recommend the following configuration for real-world deployments:

```python
# Optimal HNSW configuration for balanced performance
index_params = {
    "M": 32,                  # Balance recall and latency
    "ef_construction": 200,   # Higher quality index
    "ef_runtime": 100         # Fast search with good recall
}
```

This configuration achieves 0.94 recall with 4.3ms p95 latency, suitable for most real-time applications.
For applications with stricter latency requirements (<2ms), consider M=16 with ef_construction=100,
accepting the lower recall of 0.89. For applications requiring maximum recall (>0.95), use M=64
with ef_construction=400, but ensure adequate memory and accept higher latency.

[... 1,500 more words with additional analysis ...]

## 5. Discussion and Conclusion
Our findings demonstrate that vector search optimization is fundamentally about understanding
YOUR specific requirements and constraints. There is no one-size-fits-all configuration. The choice
between HNSW parameters depends on your specific recall requirements, latency budget, and memory constraints.
We provide a mathematical model and practical guidelines to help practitioners make informed decisions...
"""

paper_tokens = count_tokens(research_paper)
print(f"Token count: {paper_tokens:,} | Words: ~{len(research_paper.split())}")

**📊 Analysis: Research Paper Example**

**Document:** "Optimizing Vector Search Performance in Redis"

**Structure:** Abstract, Introduction, Background, Methodology, Results, Discussion

**Chunking needed?** ✅ **YES**

**Why This Document May Benefit from Chunking (Even with Large Context Windows):**

> **Note:** Modern LLMs can handle 128K+ tokens, so "fitting in context" isn't the issue. The real value of chunking is **better data modeling and retrieval precision**.

**1. Retrieval Precision vs. Recall Trade-off**

Without chunking (embed entire paper):
- Query: "What compression techniques were used?"
- Retrieved: Entire 15,000-token paper (includes Abstract, Background, Results, Discussion)
- Problem: 80% of retrieved content is irrelevant to the query
- LLM must process 15,000 tokens to find 200 tokens of relevant information

With chunking (embed by section):
- Query: "What compression techniques were used?"
- Retrieved: Methodology section (800 tokens)
- Result: 90%+ of retrieved content is directly relevant
- LLM processes 800 focused tokens with high signal-to-noise ratio

**2. Structured Content Requires Specialized Chunking**

Research papers contain heterogeneous content types that need different handling. Without specialized chunking, there will be a danger of mixing incompatible content types, chunking in the middle of tables, etc.

**Tables and Charts:**
```
Table 1: HNSW Performance Comparison
| M  | ef_construction | Recall@10 | Latency (ms) | Memory (GB) |
|----|-----------------|-----------|--------------|-------------|
| 16 | 100            | 0.89      | 2.1          | 1.2         |
| 32 | 200            | 0.94      | 4.3          | 2.1         |
| 64 | 400            | 0.97      | 8.7          | 3.8         |
```

**Best practice:** Chunk table WITH its caption and explanation:
- ✅ "Table 1 shows HNSW performance trade-offs. As M increases from 16 to 64, recall improves from 0.89 to 0.97, but latency increases from 2.1ms to 8.7ms..."
- ❌ Don't chunk table separately from context - it becomes meaningless

**Mathematical Formulas:**
```
The recall-latency trade-off can be modeled as:
Latency(M, ef) = α·M² + β·ef + γ

Where:
- M = number of connections per layer
- ef = size of dynamic candidate list
- α, β, γ = dataset-specific constants
```

**Best practice:** Chunk formula WITH its explanation and variable definitions
- ✅ Keep formula + explanation + interpretation together
- ❌ Don't separate formula from its meaning

**Code Snippets:**
```python
# Optimal HNSW configuration for our use case
index_params = {
    "M": 32,              # Balance recall and latency
    "ef_construction": 200,  # Higher quality index
    "ef_runtime": 100     # Fast search
}
```

**Best practice:** Chunk code WITH its context and rationale
- ✅ "For real-world deployment, we recommend M=32 and ef_construction=200 because..."
- ❌ Don't chunk code without explaining WHY these values

**3. Query-Specific Retrieval Patterns**

Different queries need different chunks:

| Query | Needs | Without Chunking | With Chunking |
|-------|-------|------------------|---------------|
| "What compression techniques?" | Methodology section | Entire paper (15K tokens) | Methodology (800 tokens) |
| "What were recall results?" | Results + Table 1 | Entire paper (15K tokens) | Results section (600 tokens) |
| "How does HNSW work?" | Background + Formula | Entire paper (15K tokens) | Background (500 tokens) |
| "What's the recommended config?" | Discussion + Code | Entire paper (15K tokens) | Discussion (400 tokens) |

**Impact:** 10-20x reduction in irrelevant context, leading to faster responses and better quality.

**4. Embedding Quality: Focused vs. Averaged**

**Without chunking:**
- Embedding represents "a paper about vector search, HNSW, compression, benchmarks, Redis..."
- Generic, averaged representation
- Matches weakly with specific queries

**With chunking:**
- Methodology chunk: "compression techniques, quantization, memory reduction, implementation details..."
- Results chunk: "recall metrics, latency measurements, performance comparisons, benchmark data..."
- Each embedding is focused and matches strongly with relevant queries

**💡 Key Insight:** Chunking isn't about fitting in context windows - it's about **data modeling for retrieval**. Just like you wouldn't store all customer data in one database row, you shouldn't embed all document content in one vector.

### Chunking Strategies: Engineering Trade-Offs

Once you've determined that your data needs chunking, the next question is: **How should you chunk it?**

There's no single "best" chunking strategy - the optimal approach depends on YOUR data characteristics and query patterns. Let's explore different strategies and their trade-offs.

**🔧 Using LangChain for Professional-Grade Chunking**

In this section, we'll use **LangChain's text splitting utilities** for Strategies 2 and 3. LangChain provides battle-tested, robust implementations that handle edge cases and optimize for LLM consumption.

**Why LangChain?**
- **Industry-standard**: Used by thousands of real-world applications
- **Smart boundary detection**: Respects natural text boundaries (paragraphs, sentences, words)
- **Local embeddings**: Free semantic chunking with HuggingFace models (no API costs)
- **Well-tested**: Handles edge cases (empty chunks, unicode, special characters)

We'll use:
- `RecursiveCharacterTextSplitter` (Strategy 2): Smart fixed-size chunking with boundary awareness
- `SemanticChunker` + `HuggingFaceEmbeddings` (Strategy 3): Meaning-based chunking with local models

### Strategy 1: Document-Based Chunking (Structure-Aware)

**Concept:** Split documents based on their inherent structure (sections, paragraphs, headings, and as mentioned earlier, tables, code, and formulas).

**Best for:** Structured documents with clear logical divisions (research papers, technical docs, books, etc.).

In [18]:
# Strategy 1: Document-Based Chunking
# Split research paper by sections (using markdown headers)


def chunk_by_structure(text: str, separator: str = "\n## ") -> List[str]:
    """Split text by structural markers (e.g., markdown headers)."""

    # Split by headers
    sections = text.split(separator)

    # Clean and format chunks
    chunks = []
    for i, section in enumerate(sections):
        if section.strip():
            # Add header back (except for first chunk which is title)
            if i > 0:
                chunk = "## " + section
            else:
                chunk = section
            chunks.append(chunk.strip())

    return chunks


# Apply to research paper
structure_chunks = chunk_by_structure(research_paper)

print(
    f"""📊 Strategy 1: Document-Based (Structure-Aware) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Number of chunks: {len(structure_chunks)}

Chunk breakdown:
"""
)

for i, chunk in enumerate(structure_chunks):
    chunk_tokens = count_tokens(chunk)
    # Show first 100 chars of each chunk
    preview = chunk[:300].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...\n")


📊 Strategy 1: Document-Based (Structure-Aware) Chunking
Original document: 1,035 tokens
Number of chunks: 7

Chunk breakdown:

   Chunk 1: 8 tokens - # Optimizing Vector Search Performance in Redis...

   Chunk 2: 108 tokens - ## Abstract This paper presents a comprehensive analysis of vector search optimization techniques in Redis, examining the trade-offs between search quality, latency, and memory usage. We evaluate multiple indexing strategies including HNSW and FLAT indexes across datasets ranging from 10K to 10M vec...

   Chunk 3: 98 tokens - ## 1. Introduction Vector databases have become essential infrastructure for modern AI applications, enabling semantic search, recommendation systems, and retrieval-augmented generation (RAG). Redis, traditionally known as an in-memory data structure store, has evolved to support high-performance ve...

   Chunk 4: 98 tokens - ## 2. Background and Related Work Previous work on vector search optimization has focused primarily on algorithmic 

**Strategy 1 Analysis:**

✅ **Advantages:**
- Respects document structure (sections stay together)
- Semantically coherent (each chunk is a complete section)
- Easy to implement for structured documents
- Preserves author's logical organization
- **Keeps tables, formulas, and code WITH their context** (e.g., "## 3. Performance Analysis" section includes Table 1 WITH its explanation, and "## 3.2 Mathematical Model" includes the formula WITH its variable definitions)

⚠️ **Trade-offs:**
- Variable chunk sizes (some sections longer than others)
- Requires documents to have clear structure
- May create chunks that are still too large
- Doesn't work for unstructured text

🎯 **Best for:**
- Research papers with clear sections
- Technical documentation with headers
- Books with chapters/sections
- Any markdown/HTML content with structural markers

💡 **Key Insight:**
Notice how Chunk 3 ("## 3. Performance Analysis and Results") contains Table 1 along with its explanation and interpretation. This is the correct approach - the table is meaningless without context. Similarly, the mathematical formula in section 3.2 stays with its variable definitions and interpretation. This is why structure-aware chunking is superior to fixed-size chunking for technical documents.

### Strategy 2: Fixed-Size Chunking (Token-Based)

**Concept:** Split text into chunks of a predetermined size (e.g., 512 tokens) with overlap.

**Best for:** Unstructured text, quick prototyping, when you need consistent chunk sizes.

Trade-offs:
- Ignores document structure (may split mid-sentence or mid-paragraph or mid-table)
- Can break semantic coherence
- May split important information across chunks

In [19]:
# Strategy 2: Fixed-Size Chunking (Using LangChain)
# Industry-standard approach with smart boundary detection

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create text splitter with smart boundary detection
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Target chunk size in characters
    chunk_overlap=100,  # Overlap to preserve context
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""],  # Try these in order
    is_separator_regex=False,
)

print("🔄 Running fixed-size chunking with LangChain...")
print("   Trying to split on: paragraphs → sentences → words → characters\n")

# Apply to research paper
fixed_chunks_docs = text_splitter.create_documents([research_paper])
fixed_chunks = [doc.page_content for doc in fixed_chunks_docs]

print(
    f"""📊 Strategy 2: Fixed-Size (LangChain) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Target chunk size: 800 characters (~200 words)
Overlap: 100 characters
Number of chunks: {len(fixed_chunks)}

Chunk breakdown:
"""
)

for i, chunk in enumerate(fixed_chunks[:5]):  # Show first 5
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:100].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...")

print(f"... ({len(fixed_chunks) - 5} more chunks)")

🔄 Running fixed-size chunking with LangChain...
   Trying to split on: paragraphs → sentences → words → characters

📊 Strategy 2: Fixed-Size (LangChain) Chunking
Original document: 1,035 tokens
Target chunk size: 800 characters (~200 words)
Overlap: 100 characters
Number of chunks: 8

Chunk breakdown:

   Chunk 1: 117 tokens - # Optimizing Vector Search Performance in Redis  ## Abstract This paper presents a comprehensive ana...
   Chunk 2: 98 tokens - ## 1. Introduction Vector databases have become essential infrastructure for modern AI applications,...
   Chunk 3: 134 tokens - [... 5,000 more words covering methodology, experiments, results, discussion ...]  ## 2. Background ...
   Chunk 4: 128 tokens - ## 3. Performance Analysis and Results  ### 3.1 HNSW Configuration Trade-offs  Table 1 shows the per...
   Chunk 5: 206 tokens - Table 1: HNSW Performance Comparison | M  | ef_construction | Recall@10 | Latency (ms) | Memory (GB)...
... (3 more chunks)


**Strategy 2 Analysis:**

✅ **Advantages:**
- **Respects natural boundaries**: Tries paragraphs → sentences → words → characters
- Consistent chunk sizes (predictable token usage)
- Works on any text (structured or unstructured)
- Fast processing
- **Doesn't split mid-sentence** (unless absolutely necessary)

⚠️ **Trade-offs:**
- Ignores document structure (doesn't understand sections)
- Can break semantic coherence (may split related content)
- Overlap creates redundancy (increases storage/cost)
- May split important information across chunks

🎯 **Best for:**
- Unstructured text (no clear sections)
- Quick prototyping and baselines
- When consistent chunk sizes are required
- Simple documents where structure doesn't matter

💡 **How RecursiveCharacterTextSplitter Works:**

Unlike naive fixed-size splitting, this algorithm:

1. **Tries separators in order**: `["\n\n", "\n", ". ", " ", ""]`
2. **Splits on first successful separator** that keeps chunks under target size
3. **Falls back to next separator** if chunks are still too large
4. **Preserves natural boundaries** (paragraphs > sentences > words > characters)

**Example:**
- Target: 800 characters
- First try: Split on `\n\n` (paragraphs)
- If paragraph > 800 chars: Split on `\n` (lines)
- If line > 800 chars: Split on `. ` (sentences)
- And so on...

**Why this is better than naive splitting:**
- ✅ Respects natural text boundaries
- ✅ Doesn't split mid-sentence (unless necessary)
- ✅ Maintains readability
- ✅ Better for LLM comprehension

### Strategy 3: Semantic Chunking (Meaning-Based)

**Concept:** Split text based on semantic similarity using embeddings - create new chunks when topic changes significantly.

**How it works:**
1. Split text into sentences or paragraphs
2. Generate embeddings for each segment
3. Calculate similarity between consecutive segments
4. Create chunk boundaries where similarity drops (topic shift detected)

**Best for:** Dense academic text, legal documents, narratives where semantic boundaries don't align with structure.

In [20]:
# Strategy 3: Semantic Chunking (Using LangChain)
# Industry-standard approach with local embeddings (no API costs!)

from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
import os

# Suppress tokenizer warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize local embeddings (no API costs!)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# Create semantic chunker with percentile-based breakpoint detection
semantic_chunker = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",  # Split at bottom 25% of similarities
    breakpoint_threshold_amount=25,  # 25th percentile
    buffer_size=1,  # Compare consecutive sentences
)

print("🔄 Running semantic chunking with LangChain...")
print("   Using local embeddings (sentence-transformers/all-MiniLM-L6-v2)")
print("   Breakpoint detection: 25th percentile of similarity scores\n")

# Apply to research paper
semantic_chunks_docs = semantic_chunker.create_documents([research_paper])

# Extract text from Document objects
semantic_chunks = [doc.page_content for doc in semantic_chunks_docs]

print(
    f"""📊 Strategy 3: Semantic (LangChain) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens
Breakpoint method: Percentile (25th percentile)
Number of chunks: {len(semantic_chunks)}

Chunk breakdown:
"""
)

for i, chunk in enumerate(semantic_chunks):
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:200].replace("\n", " ")
    print(f"   Chunk {i+1}: {chunk_tokens:,} tokens - {preview}...\n")

08:43:27 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


🔄 Running semantic chunking with LangChain...
   Using local embeddings (sentence-transformers/all-MiniLM-L6-v2)
   Breakpoint detection: 25th percentile of similarity scores



📊 Strategy 3: Semantic (LangChain) Chunking
Original document: 1,035 tokens
Breakpoint method: Percentile (25th percentile)
Number of chunks: 25

Chunk breakdown:

   Chunk 1: 70 tokens -  # Optimizing Vector Search Performance in Redis  ## Abstract This paper presents a comprehensive analysis of vector search optimization techniques in Redis, examining the trade-offs between search qu...

   Chunk 2: 26 tokens - Our results demonstrate that careful index configuration can improve search latency by up to 10x while maintaining 95%+ recall....

   Chunk 3: 22 tokens - We also introduce novel compression techniques that reduce memory usage by 75% with minimal impact on search quality....

   Chunk 4: 4 tokens - ## 1....

   Chunk 5: 60 tokens - Introduction Vector databases have become essential infrastructure for modern AI applications, enabling semantic search, recommendation systems, and retrieval-augmented generation (RAG). Redis, tradit...

   Chunk 6: 16 tokens - However, optimizing

**Strategy 3 Analysis:**

✅ **Advantages:**
- **Detects actual topic changes** using semantic similarity (not just structural markers)
- Preserves semantic coherence (topics stay together even without headers)
- Better retrieval quality (chunks are topically focused)
- Adapts to content (works on unstructured text)
- Reduces context loss at boundaries (doesn't split mid-topic)
- **Free and local**: Uses sentence-transformers (no API costs)

⚠️ **Trade-offs:**
- Slower processing (must compute embeddings for each sentence)
- Variable chunk sizes (depends on topic boundaries)
- Higher computational cost (embedding computation + similarity calculations)
- Requires initial model download (~90MB for all-MiniLM-L6-v2)

🎯 **Best for:**
- Dense academic papers with complex topic transitions
- Legal documents where semantic sections don't have headers
- Narratives where topics don't align with structure
- Unstructured text (emails, transcripts, conversations)
- When retrieval quality is more important than processing speed

💡 **How Percentile-Based Breakpoint Detection Works:**

Instead of using a fixed similarity threshold (e.g., 0.75), the percentile method:

1. **Computes all similarities** between consecutive sentences
2. **Calculates percentiles** of the similarity distribution
3. **Creates breakpoints** where similarity is in the bottom X percentile

**Example:**
- Similarities: [0.92, 0.88, 0.45, 0.91, 0.35, 0.89]
- 25th percentile: 0.45
- Breakpoints created at: positions 2 (0.45) and 4 (0.35)

**Why this is better than fixed threshold:**
- ✅ Adapts to document's similarity distribution
- ✅ Works across different document types
- ✅ No manual threshold tuning needed
- ✅ More robust to outliers

**Alternative Breakpoint Methods:**
- `"gradient"`: Detects sudden drops in similarity (topic shifts)
- `"standard_deviation"`: Uses statistical deviation from mean
- `"interquartile"`: Uses IQR-based outlier detection

This is fundamentally different from structure-based chunking - it detects semantic boundaries regardless of headers or formatting.

---

## Deep Dive: Document Types and Specialized Chunking Strategies

The chunking strategies we've covered (document-based, fixed-size, semantic) are foundational approaches. However, real-world documents often require specialized handling based on their content type and structure.

Let's explore three distinct document categories that require different chunking considerations:

### 1. Multi-Modal Documents: Research Papers, Books, Technical Reports

These documents contain **heterogeneous content types** that require specialized handling beyond simple text chunking.

**Content Types in Multi-Modal Documents:**

1. **Text sections** (prose, paragraphs)
   - Standard semantic or structure-aware chunking works well
   - Can use any of the strategies we've covered

2. **Mathematical formulas and equations**
   - ⚠️ **Critical:** Must be kept intact with surrounding context
   - Splitting a formula from its explanation renders it meaningless
   - Example: `Latency(M, ef) = α·M² + β·ef + γ` needs variable definitions

3. **Charts, graphs, and figures**
   - May require **image extraction** and separate processing
   - Options:
     - **Multimodal embeddings** (CLIP, GPT-4V) to embed images directly
     - **OCR** to extract text from images
     - **Caption + description** as text proxy
   - Keep figure WITH its caption and reference in text

4. **Tables**
   - ⚠️ **Critical:** Should be kept as complete units
   - Options:
     - Keep table WITH caption and explanation (best for retrieval)
     - Convert to structured format (JSON, CSV) for programmatic access
     - Use multimodal models to understand table structure
   - Never split a table across chunks

**Key Benefits of Proper Chunking for Multi-Modal Documents:**

Based on research and industry best practices:

1. **Token Limit Management**
   - Ensures content fits within model context windows
   - Prevents truncation of critical information
   - Allows processing of arbitrarily large documents

2. **Improved Retrieval Precision** ([Chroma Research, 2024](https://research.trychroma.com/evaluating-chunking))
   - Smaller, focused chunks create more accurate embeddings
   - Reduces "distractor effect" from irrelevant content
   - Improves ranking of relevant information

3. **Context Preservation**
   - Keeps semantically related information together
   - Maintains the author's logical flow
   - Preserves relationships between text, formulas, and figures

4. **Computational Efficiency**
   - Faster embedding generation (smaller chunks)
   - Reduced API costs (fewer tokens per request)
   - Better memory utilization

**Chunk Overlap: A Critical Technique**

For multi-modal documents, **chunk overlap** (10-20%) is often beneficial:

- **Prevents splitting mid-concept**: Overlap ensures related information isn't fragmented
- **Preserves context across boundaries**: Important for formulas, tables, and figures
- **Improves retrieval recall**: Increases chances of capturing relevant information

**Example:**
```
Chunk 1: "...The HNSW algorithm uses parameter M to control connectivity. [OVERLAP START] Higher M values improve recall but increase latency. [OVERLAP END]"

Chunk 2: "[OVERLAP START] Higher M values improve recall but increase latency. [OVERLAP END] Table 1 shows the trade-offs..."
```

This ensures that if a query matches "M parameter effects," both chunks can be retrieved.

### Other Effective Chunking Strategies (Beyond the Basics)

While we've covered the four main strategies, research and industry practice have identified several other effective approaches:

**1. Page-Level Chunking**

**Concept:** Use natural page boundaries as chunk boundaries.

**Best for:** Structured documents like financial reports, legal filings, academic papers

**Why it works:**
- Page boundaries often align with coherent information units
- Preserves visual layout (tables, figures stay with their page)
- Proven effective for financial documents ([NVIDIA Research, 2024](https://developer.nvidia.com/blog/finding-the-best-chunking-strategy-for-accurate-ai-responses/))

**Trade-offs:**
- Variable chunk sizes (pages vary in content density)
- May not work for digital-first documents without page concept
- Requires PDF or paginated source

**Example use case:** 10-K financial reports where each page contains related financial data, tables, and explanations.

**2. Structure-Aware (Markdown/HTML) Chunking**

**Concept:** Split by hierarchical boundaries (headings, subheadings) while preserving document structure.

**Best for:** Technical documentation, wikis, markdown-based content

**Why it works:**
- Maintains logical flow of information
- Respects author's intended organization
- Natural semantic boundaries

**Implementation:**
```python
# Split by heading hierarchy
def chunk_by_headers(markdown_text):
    # Split on ## headers (sections)
    # Keep ### subsections with their parent section
    # Preserve code blocks, tables within sections
```

**3. Recursive Chunking**

**Concept:** Split by progressively smaller separators until target size is reached.

**Separator hierarchy:** `paragraphs → sentences → words → characters`

**Why it works:**
- Preserves semantic coherence at multiple levels
- Adapts to content structure
- Avoids hard breaks mid-sentence

**This is what LangChain's `RecursiveCharacterTextSplitter` does** (Strategy 2 above).

**4. Semantic Chunking with Embeddings**

**Concept:** Use embeddings to group semantically related sentences into coherent ideas.

**Why it works:**
- Detects topic boundaries even without structural markers
- Works across structural boundaries (e.g., related content in different sections)
- Adapts to content semantics, not just structure

**This is what LangChain's `SemanticChunker` does** (Strategy 3 above).

**5. Contextual Retrieval** ([Anthropic, 2024](https://www.anthropic.com/news/contextual-retrieval))

**Concept:** Add LLM-generated context to each chunk before embedding.

**How it works:**
1. For each chunk, generate a brief context: "This chunk is from Section 3.2 discussing HNSW parameters..."
2. Prepend context to chunk before embedding
3. Embed: `[context] + [chunk]`

**Results:** Reduces retrieval failures by 49-67% (Anthropic research)

**Why it works:**
- Chunks become self-contained (don't rely on surrounding context)
- Improves embedding quality (more context = better representation)
- Helps with cross-references and dependencies

**6. Late Chunking** ([Jina AI, 2024](https://arxiv.org/abs/2409.04701))

**Concept:** Embed the entire document first, then chunk the embeddings.

**How it works:**
1. Generate embeddings for entire document using long-context model
2. Chunk the embedding space (not the text)
3. Each chunk's embedding preserves full document context

**Results:** Preserves cross-chunk context without ColBERT-level storage costs

**Why it works:**
- Embeddings capture full document context
- Chunking happens in embedding space, preserving relationships
- Best of both worlds: whole-document context + chunk-level retrieval

**7. Agentic/Proposition-Based Chunking**

**Concept:** Use LLMs to extract atomic propositions (facts) from text, then chunk by propositions.

**How it works:**
1. LLM extracts individual facts/claims from text
2. Group related propositions into chunks
3. Each chunk contains semantically related facts

**Best for:** Dense academic text, legal documents, fact-heavy content

**Trade-offs:** High computational cost (LLM calls for every document)

### 2. Legal Contracts and Clause-Level Documents

**⚠️ Important:** Legal contracts represent a **fundamentally different challenge** that goes beyond simple chunking into the realm of **complex data engineering and knowledge modeling**.

**Why Legal Documents Are Different:**

1. **Clause-Level Granularity**
   - Each clause may need to be a separate retrieval unit
   - Clauses have specific legal meanings and purposes
   - Example: "Termination Clause," "Indemnification Clause," "Force Majeure"

2. **Cross-References and Dependencies**
   - Clauses often reference other clauses: "as defined in Section 3.2"
   - Understanding one clause may require reading referenced clauses
   - Circular dependencies are common

3. **Hierarchical Structure**
   - Parent-child relationships: Sections → Subsections → Clauses → Sub-clauses
   - Inheritance of terms and conditions
   - Nested definitions and exceptions

4. **Legal Precedence and Overrides**
   - Some clauses modify or override others
   - "Notwithstanding Section 2.1..." creates dependency
   - Amendment clauses change earlier provisions

5. **Rich Metadata Requirements**
   - Clause type (termination, payment, liability, etc.)
   - Parties involved (which party has obligations)
   - Effective dates and conditions
   - Jurisdiction and governing law

**What This Requires:**

Simple chunking is **insufficient** for legal documents. You need:

**1. Advanced Data Modeling**
- **Knowledge graphs** to capture clause relationships
- **Hierarchical structures** to preserve document organization
- **Dependency tracking** for cross-references

**Example knowledge graph:**
```
Clause 3.2 (Payment Terms)
  ├─ references → Clause 1.5 (Definitions: "Net 30")
  ├─ modified_by → Clause 8.1 (Amendment: "Net 45 for Q4")
  └─ depends_on → Clause 2.1 (Delivery Conditions)
```

**2. Custom Chunking Logic**
- Domain-specific rules for legal document structure
- Clause boundary detection (not just paragraphs)
- Preservation of numbering and hierarchy

**3. Context Assembly Strategies**
- **Recursive retrieval**: When retrieving Clause 3.2, also fetch referenced clauses
- **Graph traversal**: Follow dependency links to build complete context
- **Hierarchical expansion**: Include parent section context

**Example Retrieval Flow:**
```
Query: "What are the payment terms?"

1. Retrieve: Clause 3.2 (Payment Terms)
2. Detect references: "as defined in Section 1.5"
3. Fetch: Clause 1.5 (Definitions)
4. Detect modifications: Clause 8.1 modifies 3.2
5. Fetch: Clause 8.1 (Amendment)
6. Assemble context: [3.2 + 1.5 + 8.1] with relationship metadata
```

**Research and Industry Approaches:**

- **Multi-Graph Multi-Agent Systems** ([Medium, 2024](https://medium.com/enterprise-rag/legal-document-rag-multi-graph-multi-agent-recursive-retrieval-through-legal-clauses-c90e073e0052))
  - Use multiple knowledge graphs to model different relationship types
  - Agent-based recursive retrieval through clause dependencies

- **GraphRAG for Legal Contracts** ([Neo4j, 2024](https://neo4j.com/blog/developer/agentic-graphrag-for-commercial-contracts/))
  - Build knowledge graphs from contract structure
  - Use graph queries to navigate clause relationships

**💡 Discussion Question:**

Do you agree that legal contracts represent a fundamentally different challenge that goes beyond chunking into the realm of complex data engineering and knowledge modeling?

**Our Position:** Yes, absolutely. Legal documents require:
- **Not just chunking**, but **relationship modeling**
- **Not just retrieval**, but **dependency resolution**
- **Not just embeddings**, but **structured knowledge graphs**

This is an **advanced topic beyond the scope of this module**, but it's important to recognize when simple chunking strategies are insufficient.

**Recommendation:** For legal document RAG systems:
1. Start with clause-level chunking as a baseline
2. Add metadata enrichment (clause type, parties, dates)
3. Build knowledge graphs for relationships
4. Implement recursive retrieval for dependencies
5. Consider specialized legal NLP tools (e.g., LexNLP, Blackstone)

This is a **research-level problem** that requires domain expertise in both legal document structure and advanced RAG techniques.

### 3. Transcripts and Meeting Notes: Temporal and Speaker-Based Chunking

**Document Characteristics:**
- Conversational flow with speaker changes
- Temporal structure (time-based progression)
- Topic shifts without clear structural markers
- Interruptions, overlaps, and informal language
- Context depends on who said what and when

**Examples:**
- Meeting transcripts (Zoom, Teams, Google Meet)
- Podcast transcripts
- Interview transcripts
- Customer service call recordings
- Presentation Q&A sessions

**Why Standard Chunking Fails:**

Traditional chunking strategies (fixed-size, semantic) don't account for:
- **Speaker identity**: Who said what matters for context
- **Temporal flow**: Conversations build on previous statements
- **Turn-taking**: Natural conversation boundaries
- **Topic drift**: Conversations meander without clear section breaks

**Specialized Chunking Strategies for Transcripts:**

**Strategy 1: Speaker-Based Chunking**

**Concept:** Chunk by speaker turns, keeping each speaker's complete statement together.

**Best for:** Interviews, Q&A sessions, debates

**Implementation:**
```python
def chunk_by_speaker(transcript):
    """
    Input: "Speaker A: Hello, how are you? Speaker B: I'm good, thanks!"
    Output: [
        "Speaker A: Hello, how are you?",
        "Speaker B: I'm good, thanks!"
    ]
    """
    # Split on speaker labels
    # Keep speaker identity with each chunk
    # Preserve turn-taking structure
```

**Pros:**
- Preserves speaker context
- Natural conversation boundaries
- Easy to attribute statements

**Cons:**
- Variable chunk sizes (some speakers talk more)
- May split related topics across speakers

**Strategy 2: Time-Based Chunking**

**Concept:** Chunk by time intervals (e.g., 2-minute segments).

**Best for:** Long meetings, podcasts, lectures

**Implementation:**
```python
def chunk_by_time(transcript_with_timestamps, interval_seconds=120):
    """
    Input: [(0, "Speaker A: ..."), (45, "Speaker B: ..."), (130, "Speaker A: ...")]
    Output: [
        "00:00-02:00: [Speaker A: ...] [Speaker B: ...]",
        "02:00-04:00: [Speaker A: ...]"
    ]
    """
    # Group utterances by time windows
    # Preserve timestamps for reference
```

**Pros:**
- Consistent chunk sizes
- Easy to reference ("at 15:30 in the meeting")
- Works well for long recordings

**Cons:**
- May split mid-sentence or mid-topic
- Doesn't respect conversation flow

**Strategy 3: Topic-Based Chunking (Semantic + Speaker-Aware)**

**Concept:** Detect topic shifts using embeddings + speaker changes.

**Best for:** Multi-topic meetings, panel discussions

**Implementation:**
```python
def chunk_by_topic_and_speaker(transcript):
    """
    1. Compute embeddings for each utterance
    2. Detect topic shifts (low similarity between consecutive utterances)
    3. Also create boundaries at speaker changes
    4. Merge short chunks (< min_size)
    """
    # Hybrid approach: semantic + speaker-aware
```

**Pros:**
- Respects both topic and speaker boundaries
- Most semantically coherent chunks
- Adapts to conversation structure

**Cons:**
- Computationally expensive (embeddings for each utterance)
- Requires speaker diarization

**Strategy 4: Silence-Aware Merging**

**Concept:** Use silence/pauses in audio to detect natural boundaries.

**Best for:** Presentations, lectures with clear pauses

**How it works:**
- Audio processing detects pauses (e.g., >2 seconds of silence)
- Merge utterances between pauses into chunks
- Preserves natural speaking rhythm

**Source:** VoxRAG (arXiv:2505.17326, 2025) - transcription-free RAG using silence-aware chunking

**Pros:**
- Natural conversation boundaries
- Works without speaker diarization
- Respects speaker's pacing

**Cons:**
- Requires audio access (not just transcript)
- Pauses don't always align with topic shifts

**Advanced Technique: Speaker Diarization + Chunking**

**What is Speaker Diarization?**
- Automatic detection of "who spoke when"
- Assigns speaker labels (Speaker 1, Speaker 2, etc.)
- Essential for multi-speaker transcripts

**How to combine with chunking:**
1. **Diarize audio** → identify speakers
2. **Transcribe with speaker labels** → "Speaker A: Hello..."
3. **Chunk by speaker + topic** → preserve context

**Tools:**
- **Parakeet/Whisper**: Fast transcription with speaker labels
- **pyannote.audio**: State-of-the-art speaker diarization
- **AssemblyAI**: Commercial API with built-in diarization

**Example Use Case: Meeting Minutes RAG**

**Scenario:** Build a RAG system for company meeting transcripts.

**Requirements:**
- Answer questions like "What did Sarah say about the budget?"
- Retrieve action items assigned to specific people
- Find discussions about specific topics

**Recommended Approach:**
1. **Transcribe with speaker diarization** (Whisper + pyannote)
2. **Chunk by speaker + topic** (hybrid approach)
3. **Enrich with metadata**:
   - Speaker name
   - Timestamp
   - Meeting title/date
   - Detected topics (using LLM)
4. **Store chunks with metadata** in vector DB
5. **Implement metadata filtering** (e.g., "speaker:Sarah AND topic:budget")

**Metadata Enrichment Example:**
```python
chunk = {
    "text": "Sarah: I think we should increase the marketing budget by 20%...",
    "metadata": {
        "speaker": "Sarah Johnson",
        "timestamp": "00:15:30",
        "meeting": "Q4 Planning - 2025-04-15",
        "topic": "budget",
        "action_item": False
    }
}
```

**💡 Key Insight:** Transcripts require **context-aware chunking** that preserves:
- **Who** said it (speaker identity)
- **When** they said it (temporal context)
- **What** they were discussing (topic/semantic content)

Simple text chunking loses this critical context.

**Recommendation:** For transcript RAG systems:
1. Start with speaker-based chunking as baseline
2. Add speaker diarization for multi-speaker content
3. Enrich chunks with metadata (speaker, time, topic)
4. Consider hybrid chunking (speaker + semantic) for complex conversations
5. Use metadata filtering to improve retrieval precision

This is an **emerging area** with active research (VoxRAG, 2025) exploring transcription-free approaches.

### 4. Implementation Guidance: Choosing Your Chunking Strategy

The optimal chunking strategy requires **experimentation** based on your specific context. There's no universal "best practice."

**Factors to Consider:**

1. **Specific Use Case and Query Patterns**
   - What questions will users ask?
   - Do they need summaries or specific facts?
   - Are queries broad or narrow?

2. **Document Type and Structure**
   - Structured (sections, headers) or unstructured (prose)?
   - Multi-modal (text + images + tables) or text-only?
   - Single-topic or multi-topic documents?

3. **Models Being Used**
   - **Embedding model**: Context window size, quality on long vs. short text
   - **LLM**: Context window, attention patterns, cost per token
   - **Multimodal capabilities**: Can it handle images, tables?

4. **Performance Metrics**
   - **Retrieval precision**: Are you getting the right chunks?
   - **Answer quality**: Are LLM responses accurate and complete?
   - **Latency**: How fast do you need results?
   - **Cost**: Token usage, API calls, storage

**Experimentation Framework:**

```python
# 1. Define your test queries
test_queries = [
    "What are the payment terms?",
    "Explain the HNSW algorithm",
    "What compression techniques were used?"
]

# 2. Try multiple chunking strategies
strategies = [
    ("no_chunking", whole_document),
    ("document_based", chunk_by_structure),
    ("fixed_size_512", fixed_chunking_512),
    ("fixed_size_1024", fixed_chunking_1024),
    ("semantic", semantic_chunking),
]

# 3. Measure performance
for strategy_name, chunking_fn in strategies:
    chunks = chunking_fn(documents)

    # Measure retrieval precision
    precision = evaluate_retrieval(chunks, test_queries)

    # Measure answer quality (LLM evaluation)
    quality = evaluate_answers(chunks, test_queries, llm)

    # Measure efficiency
    avg_chunk_size = np.mean([len(c) for c in chunks])
    total_tokens = sum([count_tokens(c) for c in chunks])

    print(f"{strategy_name}: Precision={precision:.2f}, Quality={quality:.2f}, Tokens={total_tokens}")
```

**Key Metrics to Track:**

| Metric | What It Measures | How to Evaluate |
|--------|------------------|-----------------|
| **Retrieval Precision** | Are the right chunks retrieved? | Manual review or automated eval (LLM-as-judge) |
| **Answer Quality** | Are LLM responses accurate? | Human evaluation or LLM-based scoring |
| **Token Efficiency** | How many tokens per query? | Count tokens in retrieved chunks |
| **Latency** | How fast is retrieval? | Measure end-to-end query time |
| **Coverage** | Do chunks cover all important info? | Check if key facts are retrievable |

**Iterative Improvement Process:**

1. **Start simple**: Begin with fixed-size or document-based chunking
2. **Measure baseline**: Establish performance metrics
3. **Identify failures**: Where does retrieval fail? Where are answers wrong?
4. **Hypothesize improvements**: "Tables are being split" → Try structure-aware chunking
5. **Test and compare**: Measure impact of changes
6. **Iterate**: Repeat until performance is acceptable

**Common Failure Patterns and Solutions:**

| Problem | Likely Cause | Solution |
|---------|--------------|----------|
| Tables split across chunks | Fixed-size chunking | Use structure-aware chunking |
| Formulas without context | Naive chunking | Keep formulas with explanations |
| Missing cross-references | Single-chunk retrieval | Implement recursive retrieval |
| Generic answers | Chunks too large | Reduce chunk size or use semantic chunking |
| Incomplete answers | Chunks too small | Increase chunk size or add overlap |

**💡 Final Insight:**

Chunking is **data modeling for retrieval**. Just like database schema design, there's no one-size-fits-all solution. The "best" chunking strategy is the one that:
- Matches your document structure
- Aligns with your query patterns
- Meets your performance requirements
- Balances complexity with maintainability

**Experiment, measure, iterate.**

### Strategy 4: Hierarchical Chunking (Multi-Level)

**Concept:** Create multiple levels of chunks - large chunks for overview, small chunks for details.

**Best for:** Very large documents where users need both high-level summaries and specific details.

In [21]:
# Strategy 4: Hierarchical Chunking


def chunk_hierarchically(text: str) -> Dict[str, List[str]]:
    """
    Create multiple levels of chunks.
    Level 1: Large sections (by ## headers)
    Level 2: Subsections (by paragraphs within sections)
    """

    # Level 1: Split by major sections
    level1_chunks = chunk_by_structure(text, separator="\n## ")

    # Level 2: Further split large sections into paragraphs
    level2_chunks = []
    for section in level1_chunks:
        # If section is large, split into paragraphs
        if count_tokens(section) > 400:
            paragraphs = [
                p.strip() for p in section.split("\n\n") if p.strip() and len(p) > 50
            ]
            level2_chunks.extend(paragraphs)
        else:
            level2_chunks.append(section)

    return {
        "level1": level1_chunks,  # Large sections
        "level2": level2_chunks,  # Smaller subsections
    }


# Apply to research paper
hierarchical_chunks = chunk_hierarchically(research_paper)

print(
    f"""📊 Strategy 4: Hierarchical (Multi-Level) Chunking
{'=' * 80}
Original document: {paper_tokens:,} tokens

Level 1 (Sections): {len(hierarchical_chunks['level1'])} chunks
"""
)

for i, chunk in enumerate(hierarchical_chunks["level1"]):
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:80].replace("\n", " ")
    print(f"   L1-{i+1}: {chunk_tokens:,} tokens - {preview}...")

print(
    f"""
Level 2 (Subsections): {len(hierarchical_chunks['level2'])} chunks
"""
)

for i, chunk in enumerate(hierarchical_chunks["level2"][:5]):  # Show first 5
    chunk_tokens = count_tokens(chunk)
    preview = chunk[:80].replace("\n", " ")
    print(f"   L2-{i+1}: {chunk_tokens:,} tokens - {preview}...")

print(f"... ({len(hierarchical_chunks['level2']) - 5} more L2 chunks)")

📊 Strategy 4: Hierarchical (Multi-Level) Chunking
Original document: 1,035 tokens

Level 1 (Sections): 7 chunks

   L1-1: 8 tokens - # Optimizing Vector Search Performance in Redis...
   L1-2: 108 tokens - ## Abstract This paper presents a comprehensive analysis of vector search optimi...
   L1-3: 98 tokens - ## 1. Introduction Vector databases have become essential infrastructure for mod...
   L1-4: 98 tokens - ## 2. Background and Related Work Previous work on vector search optimization ha...
   L1-5: 464 tokens - ## 3. Performance Analysis and Results  ### 3.1 HNSW Configuration Trade-offs  T...
   L1-6: 187 tokens - ## 4. Implementation Recommendations  Based on our findings, we recommend the fo...
   L1-7: 73 tokens - ## 5. Discussion and Conclusion Our findings demonstrate that vector search opti...

Level 2 (Subsections): 13 chunks

   L2-1: 8 tokens - # Optimizing Vector Search Performance in Redis...
   L2-2: 108 tokens - ## Abstract This paper presents a comprehensive analysi

**Strategy 4 Analysis:**

✅ **Advantages:**
- Supports both overview and detailed queries
- Flexible retrieval (can search at different levels)
- Preserves document hierarchy
- Better for complex documents

⚠️ **Trade-offs:**
- More complex to implement and maintain
- Requires more storage (multiple levels)
- Need strategy to choose which level to search
- Higher indexing cost

🎯 **Best for:**
- Very large documents (textbooks, manuals)
- When users need both summaries and details
- Technical documentation with nested structure
- Legal contracts with sections and subsections

💡 **Retrieval Strategy:**
- Start with Level 1 for overview
- If user needs more detail, retrieve Level 2 chunks
- Can combine: "Show section summary + relevant details"

### Comparing Chunking Strategies: Decision Framework

Now let's compare all four strategies side-by-side:

In [22]:
print(
    f"""
{'=' * 80}
CHUNKING STRATEGY COMPARISON
{'=' * 80}

Document: Research Paper ({paper_tokens:,} tokens)

Strategy              | Chunks | Avg Size | Complexity | Best For
--------------------- | ------ | -------- | ---------- | --------
Document-Based        | {len(structure_chunks):>6} | {sum(count_tokens(c) for c in structure_chunks) // len(structure_chunks):>8} | Low        | Structured docs
Fixed-Size            | {len(fixed_chunks):>6} | {sum(count_tokens(c) for c in fixed_chunks) // len(fixed_chunks):>8} | Low        | Unstructured text
Semantic              | {len(semantic_chunks):>6} | {sum(count_tokens(c) for c in semantic_chunks) // len(semantic_chunks):>8} | High       | Dense academic text
Hierarchical (L1)     | {len(hierarchical_chunks['level1']):>6} | {sum(count_tokens(c) for c in hierarchical_chunks['level1']) // len(hierarchical_chunks['level1']):>8} | Medium     | Large complex docs
Hierarchical (L2)     | {len(hierarchical_chunks['level2']):>6} | {sum(count_tokens(c) for c in hierarchical_chunks['level2']) // len(hierarchical_chunks['level2']):>8} | Medium     | Large complex docs

{'=' * 80}
"""
)


CHUNKING STRATEGY COMPARISON

Document: Research Paper (1,035 tokens)

Strategy              | Chunks | Avg Size | Complexity | Best For
--------------------- | ------ | -------- | ---------- | --------
Document-Based        |      7 |      148 | Low        | Structured docs
Fixed-Size            |      8 |      140 | Low        | Unstructured text
Semantic              |     25 |       41 | High       | Dense academic text
Hierarchical (L1)     |      7 |      148 | Medium     | Large complex docs
Hierarchical (L2)     |     13 |       76 | Medium     | Large complex docs




### YOUR Chunking Decision Framework

Chunking strategy is a **design choice** that depends on your specific context. There's no universal "correct" chunk size.

**Step 1: Start with Document Type**

| Document Type | Default Approach | Reasoning |
|---------------|------------------|----------|
| **Structured records** (courses, products, FAQs) | Don't chunk | Natural boundaries already exist |
| **Long-form text** (papers, books, docs) | Consider chunking | May need retrieval precision |
| **PDFs with visual layout** | Page-level | Preserves tables, figures |
| **Code** | Function/class boundaries | Semantic structure matters |

**Step 2: Evaluate These Factors**

1. **Semantic completeness:** Is each item self-contained?
   - ✅ Yes → Don't chunk (preserve natural boundaries)
   - ❌ No → Consider chunking strategy

2. **Query patterns:** What will users ask?
   - Specific facts → Smaller, focused chunks help
   - Summaries/overviews → Larger chunks or hierarchical
   - Mixed → Consider hierarchical approach

3. **Topic density:** How many distinct topics per document?
   - Single topic → Whole-document embedding often works
   - Multiple distinct topics → Chunking may improve precision

4. **Embedding model:** What's your model's context window?
   - Modern models (jina-v2, etc.) support 8K+ tokens
   - Model limits are often less constraining than retrieval precision needs

**Step 3: Consider Constraints**

- **Latency:** Simple approaches (fixed-size, no chunking) are faster
- **Quality:** Semantic chunking adds cost but may improve relevance
- **Complexity:** More sophisticated ≠ better for your use case

**Example Decisions:**

| Domain | Data Characteristics | Decision | Why |
|--------|---------------------|----------|-----|
| **Course Catalog** | Small, self-contained records | **Don't chunk** | Each course is a complete retrieval unit |
| **Research Papers** | Multi-section, dense topics | Document-Based | Sections are natural semantic units |
| **Support Tickets** | Single issue per ticket | **Don't chunk** | Already at optimal granularity |
| **Legal Contracts** | Nested structure, many clauses | Hierarchical | Need both overview and clause-level detail |
| **Product Docs** | Mixed structure, varied content | Fixed-Size or Semantic | Balance simplicity with quality |

> 💡 **Key Takeaway:** Ask "What is my natural retrieval unit?" before deciding on a chunking strategy. For many structured data use cases, the answer is "don't chunk."

---

## Summary: Chunking is Subjective, Document-Dependent, and Requires Experimentation

**What We've Learned:**

Chunking is **not a one-size-fits-all solution**. The optimal strategy depends entirely on:

1. **Document Type and Structure**
   - Research papers → Structure-aware or hybrid (summary + sections)
   - Legal contracts → Knowledge graphs + recursive retrieval (beyond simple chunking)
   - Transcripts → Speaker-aware + temporal chunking
   - Structured records → Often don't chunk at all

2. **Content Heterogeneity**
   - Text-only → Standard chunking strategies work
   - Multimodal (tables, figures, equations, code) → Specialized handling required
   - Keep content WITH its context (table + caption, formula + definitions)

3. **Query Patterns**
   - Overview queries → Larger chunks or hierarchical (summary level)
   - Specific facts → Smaller, focused chunks
   - Mixed queries → Hybrid approach (multiple retrieval levels)

4. **Models Being Used**
   - Embedding model capabilities (context window, quality on long vs. short text)
   - LLM context window and attention patterns
   - Multimodal capabilities (vision models for figures)

5. **Performance Requirements**
   - Retrieval precision (are you getting the right chunks?)
   - Answer quality (are LLM responses accurate?)
   - Latency (how fast do you need results?)
   - Cost (token usage, API calls, storage)

**Key Insights from Research:**

- **49-67% improvement** in retrieval with contextual retrieval (Anthropic, 2024)
- **Chunk overlap (10-20%)** preserves context across boundaries
- **Lost in the Middle** and **Context Rot** show that structure matters more than fitting everything in
- **Semantic chunking doesn't always outperform fixed-size** - experimentation is required

**Three Document Types, Three Different Approaches:**

| Document Type | Key Challenge | Recommended Approach |
|---------------|---------------|----------------------|
| **Research Papers & Long-Form** | Multimodal content (text, tables, figures, equations) | Hybrid: summary + section-based + multimodal handling |
| **Legal Contracts** | Cross-references, hierarchical dependencies, legal precedence | Knowledge graphs + recursive retrieval (advanced topic) |
| **Transcripts & Meetings** | Speaker identity, temporal flow, topic drift | Speaker-aware + time-based + metadata enrichment |

**The Expert's Approach:**

1. **Analyze your documents** - What content types? What structure?
2. **Understand your queries** - What will users ask? How specific?
3. **Start simple** - Fixed-size or structure-aware chunking as baseline
4. **Measure performance** - Retrieval precision, answer quality, latency
5. **Identify failures** - Where does retrieval fail? Where are answers wrong?
6. **Iterate and improve** - Test alternatives, measure impact, refine
7. **Consider alternatives** - Sometimes chunking isn't the answer (hierarchical retrieval, knowledge graphs)

**Critical Questions to Ask:**

- **Is chunking even necessary?** (For structured records, often NO)
- **What is my natural retrieval unit?** (Course, section, clause, speaker turn?)
- **What content types need special handling?** (Tables, formulas, figures, code?)
- **What are my query patterns?** (Overview vs. specific facts?)
- **What are my performance constraints?** (Latency, cost, quality trade-offs?)

**Final Insight:**

Chunking is **data modeling for retrieval**. Just like database schema design, there's no universal solution. The "best" chunking strategy is the one that:
- Matches your document structure
- Aligns with your query patterns
- Meets your performance requirements
- Balances complexity with maintainability

**Experiment, measure, iterate.** This is engineering, not magic.


---

## Part 5: Building Practical Context Pipelines

Now that you understand data transformation and chunking, let's discuss how to build reusable pipelines.

### Three Pipeline Architectures

There are three main approaches to context preparation in real-world applications:

### Architecture 1: Request-Time Processing

**Concept:** Transform data on-the-fly when a query arrives.

```
User Query → Retrieve Raw Data → Transform → Chunk (if needed) → Embed → Search → Return Context
```

**Pros:**
- ✅ Always up-to-date (no stale data)
- ✅ No pre-processing required
- ✅ Simple to implement

**Cons:**
- ❌ Higher latency (processing happens during request)
- ❌ Repeated work (same transformations for every query)
- ❌ Not suitable for large datasets

**Best for:**
- Small datasets (< 1,000 documents)
- Frequently changing data
- Simple transformations

### Architecture 2: Batch Processing

**Concept:** Pre-process all data in batches (nightly, weekly) and store results.

```
[Scheduled Job]
Raw Data → Extract → Clean → Transform → Chunk → Embed → Store in Vector DB

[Query Time]
User Query → Search Vector DB → Return Pre-Processed Context
```

**Pros:**
- ✅ Fast query time (all processing done ahead)
- ✅ Efficient (process once, use many times)
- ✅ Can use expensive transformations (LLM-based chunking, semantic analysis)

**Cons:**
- ❌ Data can be stale (until next batch run)
- ❌ Requires scheduling infrastructure
- ❌ Higher storage costs (store processed data)

**Best for:**
- Large datasets (> 10,000 documents)
- Infrequently changing data (daily/weekly updates)
- Complex transformations (semantic chunking, LLM summaries)

### Architecture 3: Event-Driven Processing

**Concept:** Process data as it changes (real-time updates).

```
Data Change Event → Trigger Pipeline → Extract → Clean → Transform → Chunk → Embed → Update Vector DB

[Query Time]
User Query → Search Vector DB → Return Context
```

**Pros:**
- ✅ Always up-to-date (real-time)
- ✅ Fast query time (pre-processed)
- ✅ Efficient (only process changed data)

**Cons:**
- ❌ Complex infrastructure (event streams, queues)
- ❌ Requires change detection
- ❌ Higher operational complexity

**Best for:**
- Real-time data (news, social media, live updates)
- Large datasets that change frequently
- When both freshness and speed are critical

### Choosing YOUR Pipeline Architecture

Use this decision tree:

**Question 1: How often does your data change?**
- Real-time (seconds/minutes) → Event-Driven
- Frequently (hourly/daily) → Batch or Event-Driven
- Infrequently (weekly/monthly) → Batch
- Rarely (manual updates) → Request-Time or Batch

**Question 2: How large is your dataset?**
- Small (< 1,000 docs) → Request-Time
- Medium (1,000-100,000 docs) → Batch
- Large (> 100,000 docs) → Batch or Event-Driven

**Question 3: What are your latency requirements?**
- Real-time (< 100ms) → Batch or Event-Driven (pre-processed)
- Interactive (< 1s) → Any approach
- Batch queries → Request-Time acceptable

**Question 4: How complex are your transformations?**
- Simple (cleaning, formatting) → Any approach
- Moderate (chunking, basic NLP) → Batch or Event-Driven
- Complex (LLM-based, semantic analysis) → Batch (pre-compute)

**Example Decision:**

For Redis University:
- ✅ **Data changes:** Weekly (new courses added)
- ✅ **Dataset size:** 100-500 courses (medium)
- ✅ **Latency:** Interactive (< 1s acceptable)
- ✅ **Transformations:** Moderate (structured views + embeddings)
- ✅ **Decision:** **Batch Processing** (weekly job to rebuild catalog + embeddings)

### Example: Batch Processing Pipeline for Redis University

In [23]:
# Example: Batch Processing Pipeline
# This would run as a scheduled job (e.g., weekly)


async def batch_process_courses():
    """
    Batch processing pipeline for Redis University courses.
    Runs weekly to update catalog and embeddings.
    """

    print("=" * 80)
    print("BATCH PROCESSING PIPELINE - Redis University Courses")
    print("=" * 80)

    # Step 1: Extract
    print("\n[Step 1/5] Extracting course data...")
    all_courses = await course_manager.get_all_courses()
    print(f"   ✅ Extracted {len(all_courses)} courses")

    # Show sample raw data
    if all_courses:
        sample = all_courses[0]
        print(f"\n   📄 Sample raw course:")
        print(f"      {sample.course_code}: {sample.title}")
        print(f"      Department: {sample.department}, Credits: {sample.credits}, Level: {sample.difficulty_level.value}")

    # Step 2: Clean
    print("\n[Step 2/5] Cleaning data...")
    # Remove test courses, validate fields, etc.
    cleaned_courses = [
        c for c in all_courses if c.course_code.startswith(("RU", "CS", "MATH"))
    ]
    print(
        f"   ✅ Cleaned: {len(cleaned_courses)} courses (removed {len(all_courses) - len(cleaned_courses)} test courses)"
    )

    # Show what was filtered out
    removed_courses = [c for c in all_courses if not c.course_code.startswith(("RU", "CS", "MATH"))]
    if removed_courses:
        print(f"\n   📄 Example removed course:")
        print(f"      🗑️  {removed_courses[0].course_code}: {removed_courses[0].title} (filtered out)")

    # Step 3: Transform
    print("\n[Step 3/5] Transforming to LLM-friendly format...")
    transformed_courses = [transform_course_to_text(c) for c in cleaned_courses]
    total_tokens = sum(count_tokens(t) for t in transformed_courses)
    print(
        f"   ✅ Transformed: {len(transformed_courses)} courses ({total_tokens:,} total tokens)"
    )

    # Show before/after transformation
    if cleaned_courses and transformed_courses:
        print(f"\n   📄 Transformation example:")
        print(f"      Before: {cleaned_courses[0].course_code} (Course object)")
        print(f"      After (LLM-friendly text):")
        preview = transformed_courses[0].replace('\n', '\n      ')
        print(f"      {preview[:250]}...")

    # Step 4: Create Structured Views
    print("\n[Step 4/5] Creating structured catalog view...")
    catalog_view = await create_catalog_view()
    catalog_tokens = count_tokens(catalog_view)
    redis_client.set("course_catalog_view", catalog_view)
    redis_client.set("course_catalog_view:updated", "2024-09-30")
    print(f"   ✅ Created catalog view ({catalog_tokens:,} tokens)")
    print(f"   ✅ Cached in Redis")

    # Show catalog structure
    print(f"\n   📄 Catalog view structure:")
    catalog_preview = catalog_view[:300].replace('\n', '\n      ')
    print(f"      {catalog_preview}...")

    # Step 5: Store (in production, would also create embeddings and store in vector DB)
    print("\n[Step 5/5] Storing processed data...")
    for i, (course, text) in enumerate(zip(cleaned_courses, transformed_courses)):
        key = f"course:processed:{course.course_code}"
        redis_client.set(key, text)
    print(f"   ✅ Stored {len(cleaned_courses)} processed courses in Redis")

    # Show storage example
    if cleaned_courses:
        print(f"\n   📄 Storage example:")
        print(f"      Key: course:processed:{cleaned_courses[0].course_code}")
        print(f"      Value: {transformed_courses[0][:100]}...")

    print("\n" + "=" * 80)
    print("BATCH PROCESSING COMPLETE")
    print("=" * 80)
    print(
        f"""
Summary:
- Courses processed: {len(cleaned_courses)}
- Total tokens: {total_tokens:,}
- Catalog view tokens: {catalog_tokens:,}
- Storage: Redis
- Next run: 2024-10-07 (weekly)
"""
    )


# Run the batch pipeline
await batch_process_courses()

BATCH PROCESSING PIPELINE - Redis University Courses

[Step 1/5] Extracting course data...


08:43:29 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


   ✅ Extracted 50 courses

   📄 Sample raw course:
      CS004: Database Systems
      Department: Computer Science, Credits: 3, Level: intermediate

[Step 2/5] Cleaning data...
   ✅ Cleaned: 20 courses (removed 30 test courses)

   📄 Example removed course:
      🗑️  BUS033: Marketing Strategy (filtered out)

[Step 3/5] Transforming to LLM-friendly format...
   ✅ Transformed: 20 courses (1,087 total tokens)

   📄 Transformation example:
      Before: CS004 (Course object)
      After (LLM-friendly text):
      CS004: Database Systems
      Department: Computer Science
      Credits: 3
      Level: intermediate
      Format: online
      Instructor: John Zamora
      Description: Design and implementation of database systems. SQL, normalization, transaction...

[Step 4/5] Creating structured catalog view...
   ✅ Created catalog view (585 tokens)
   ✅ Cached in Redis

   📄 Catalog view structure:
      # Redis University Course Catalog
      
      ## Business (10 courses)
      - BUS03

---

## Summary and Key Takeaways

### What You Learned

**1. Context is Data - and Data Requires Engineering**
- Context isn't just "data you feed to an LLM"
- It requires systematic transformation: Raw → Clean → Transform → Optimize → Store
- Engineering discipline: requirements analysis, design decisions, quality metrics, testing

**2. The Data Engineering Pipeline**
- Extract: Get raw data from sources
- Clean: Remove noise, fix inconsistencies
- Transform: Structure for LLM consumption
- Optimize: Reduce tokens, improve clarity
- Store: Choose storage strategy (RAG, Views, Hybrid)

**3. Three Engineering Approaches**
- **RAG:** Semantic search for relevant data (good for specific queries)
- **Structured Views:** Pre-computed summaries (excellent for overviews)
- **Hybrid:** Combine both (best for real-world use)

**4. Chunking is a Design Choice, Not a Default**
- **"Don't chunk"** is a valid strategy for structured data (courses, products, FAQs)
- Consider chunking when documents have multiple distinct topics or need retrieval precision
- Four strategies when chunking: Document-Based, Fixed-Size, Semantic, Hierarchical
- Emerging approaches: Late Chunking, Contextual Retrieval, Hybrid Search
- Choose based on YOUR data type, query patterns, and application requirements

**5. Context Pipeline Architectures**
- **Request-Time:** Process on-the-fly (simple, always fresh, higher latency)
- **Batch:** Pre-process in batches (fast queries, can be stale)
- **Event-Driven:** Process on changes (real-time, complex infrastructure)

### The Engineering Mindset

Every decision should be based on **YOUR specific requirements:**

1. **Analyze YOUR data:** Size, structure, update frequency, topic density
2. **Analyze YOUR queries:** Specific vs. overview, single vs. cross-section
3. **Analyze YOUR constraints:** Token budget, latency, quality requirements
4. **Make informed decisions:** Choose approaches that match YOUR needs
5. **Measure and iterate:** Test with real queries, measure quality, optimize

**Remember:** There is no "best practice" that works for everyone. Context engineering is about making deliberate, informed choices based on YOUR domain, application, and constraints.

---

## Part 6: Quality Optimization - Measuring and Improving Context

### The Systematic Optimization Process

Now that you understand data engineering and context pipelines, let's learn how to systematically optimize context quality.

**The Process:**
```
1. Define Quality Metrics (domain-specific)
   ↓
2. Establish Baseline (measure current performance)
   ↓
3. Experiment (try different approaches)
   ↓
4. Measure (compare against metrics)
   ↓
5. Iterate (refine based on results)
```

---

### Step 1: Define Quality Metrics for YOUR Domain

**The Problem with Generic Metrics:**

Don't aim for "95% accuracy on benchmark X" - that benchmark wasn't designed for YOUR domain.

**DO this instead:** Define what "quality" means for YOUR domain, then measure it.

### The Four Quality Dimensions

Every context engineering solution should be evaluated across four dimensions:

1. **Relevance** - Does context include information needed to answer the query?
2. **Completeness** - Does context include ALL necessary information?
3. **Efficiency** - Is context optimized for token usage?
4. **Accuracy** - Is context factually correct and up-to-date?

Different domains prioritize these differently.

---

### Example: Quality Metrics for Redis University Course Advisor

Let's define specific, measurable quality metrics for our course advisor domain.

In [24]:
# Define domain-specific quality metrics

quality_metrics = {
    "Relevance": {
        "definition": "Does context include courses relevant to the user's query?",
        "metric": "% of queries where retrieved courses match query intent",
        "measurement": "Manual review of 50 sample queries",
        "target": ">90%",
        "why_important": "Irrelevant courses waste tokens and confuse users",
    },
    "Completeness": {
        "definition": "Does context include all information needed to answer?",
        "metric": "% of responses that mention all prerequisites when asked",
        "measurement": "Automated check: parse response for prerequisite mentions",
        "target": "100%",
        "why_important": "Missing prerequisites leads to hallucinations",
    },
    "Efficiency": {
        "definition": "Is context optimized for token usage?",
        "metric": "Average tokens per query",
        "measurement": "Token counter on all context strings",
        "target": "<5,000 tokens",
        "why_important": "Exceeding budget increases cost and latency",
    },
    "Accuracy": {
        "definition": "Is context factually correct and up-to-date?",
        "metric": "% of responses with correct course information",
        "measurement": "Manual review against course database",
        "target": ">95%",
        "why_important": "Incorrect information damages trust",
    },
}

print(
    """QUALITY METRICS FOR REDIS UNIVERSITY COURSE ADVISOR
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━"""
)
for dimension, details in quality_metrics.items():
    print(
        f"""
{dimension}:
  Definition: {details['definition']}
  Metric: {details['metric']}
  How to measure: {details['measurement']}
  Target: {details['target']}
  Why important: {details['why_important']}"""
    )
print("\n" + "━" * 80)

QUALITY METRICS FOR REDIS UNIVERSITY COURSE ADVISOR
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Relevance:
  Definition: Does context include courses relevant to the user's query?
  Metric: % of queries where retrieved courses match query intent
  How to measure: Manual review of 50 sample queries
  Target: >90%
  Why important: Irrelevant courses waste tokens and confuse users

Completeness:
  Definition: Does context include all information needed to answer?
  Metric: % of responses that mention all prerequisites when asked
  How to measure: Automated check: parse response for prerequisite mentions
  Target: 100%
  Why important: Missing prerequisites leads to hallucinations

Efficiency:
  Definition: Is context optimized for token usage?
  Metric: Average tokens per query
  How to measure: Token counter on all context strings
  Target: <5,000 tokens
  Why important: Exceeding budget increases cost and latency

Accuracy:
  Definition: Is context fact

### Key Insight: Metrics Must Be Domain-Specific

Notice how these metrics are specific to the course advisor domain:

**Relevance metric:**
- ❌ Generic: "Cosine similarity > 0.8"
- ✅ Domain-specific: "Retrieved courses match query intent"

**Completeness metric:**
- ❌ Generic: "Context includes top-5 search results"
- ✅ Domain-specific: "All prerequisites mentioned when asked"

**Efficiency metric:**
- ❌ Generic: "Minimize tokens"
- ✅ Domain-specific: "<5,000 tokens (fits our budget)"

**Accuracy metric:**
- ❌ Generic: "95% on MMLU benchmark"
- ✅ Domain-specific: "Correct course information vs. database"

**Your metrics should reflect YOUR domain's requirements, not generic benchmarks.**

---

### Step 2-5: Baseline → Experiment → Measure → Iterate

Let's demonstrate the optimization process with a concrete example.

**Scenario:** We want to optimize our hybrid approach (catalog overview + RAG) to meet all quality targets.

In [25]:
# Step 2: Establish Baseline (Hybrid Approach from Part 3)

# Sample query
test_query = "What machine learning courses are available for beginners?"

# Hybrid approach: Catalog overview + RAG
catalog_overview = """Redis University Course Catalog Overview:

Computer Science Department:
- RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)
- RU201: Redis for Python Developers (Intermediate, 6-8 hours)
- RU301: Vector Similarity Search with Redis (Advanced, 8-10 hours)

Data Science Department:
- RU401: Machine Learning with Redis (Intermediate, 10-12 hours)
- RU402: Real-Time Analytics with Redis (Advanced, 8-10 hours)
"""

# RAG: Get specific courses
rag_results = await course_manager.search_courses(test_query, limit=2)
rag_context = "\n\n".join(
    [
        f"""{course.course_code}: {course.title} ({course.difficulty_level.value})
Description: {course.description}
Prerequisites: {', '.join([p.course_code for p in course.prerequisites]) if course.prerequisites else 'None'}"""
        for course in rag_results
    ]
)

# Combined context
baseline_context = f"""{catalog_overview}

Detailed Course Information:
{rag_context}"""

baseline_tokens = count_tokens(baseline_context)

print(
    f"""BASELINE (Hybrid Approach):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Tokens: {baseline_tokens:,}

Context:
{baseline_context}
"""
)

08:43:29 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


BASELINE (Hybrid Approach):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Tokens: 177

Context:
Redis University Course Catalog Overview:

Computer Science Department:
- RU101: Introduction to Redis Data Structures (Beginner, 4-6 hours)
- RU201: Redis for Python Developers (Intermediate, 6-8 hours)
- RU301: Vector Similarity Search with Redis (Advanced, 8-10 hours)

Data Science Department:
- RU401: Machine Learning with Redis (Intermediate, 10-12 hours)
- RU402: Real-Time Analytics with Redis (Advanced, 8-10 hours)


Detailed Course Information:
CS007: Machine Learning (advanced)
Description: Introduction to machine learning algorithms and applications. Supervised and unsupervised learning, neural networks.
Prerequisites: None

MATH022: Linear Algebra (intermediate)
Description: Vector spaces, matrices, eigenvalues, and linear transformations. Essential for data science and engineering.
Prerequisites: None



In [26]:
# Step 3: Experiment - Try optimized version

# Optimization: Reduce catalog overview to just relevant departments
optimized_catalog = """Redis University - Relevant Departments:

Data Science:
- RU401: Machine Learning with Redis (Intermediate)
- RU402: Real-Time Analytics (Advanced)

Computer Science:
- RU301: Vector Search (Advanced)
"""

optimized_context = f"""{optimized_catalog}

{rag_context}"""

optimized_tokens = count_tokens(optimized_context)

print(
    f"""EXPERIMENT (Optimized Hybrid):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Tokens: {optimized_tokens:,}

Context:
{optimized_context}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Token Reduction: {baseline_tokens - optimized_tokens:,} tokens ({((baseline_tokens - optimized_tokens) / baseline_tokens * 100):.1f}% reduction)
"""
)

EXPERIMENT (Optimized Hybrid):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Tokens: 111

Context:
Redis University - Relevant Departments:

Data Science:
- RU401: Machine Learning with Redis (Intermediate)
- RU402: Real-Time Analytics (Advanced)

Computer Science:
- RU301: Vector Search (Advanced)


CS007: Machine Learning (advanced)
Description: Introduction to machine learning algorithms and applications. Supervised and unsupervised learning, neural networks.
Prerequisites: None

MATH022: Linear Algebra (intermediate)
Description: Vector spaces, matrices, eigenvalues, and linear transformations. Essential for data science and engineering.
Prerequisites: None

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Token Reduction: 66 tokens (37.3% reduction)



In [27]:
# Step 4: Measure - Compare responses

# Baseline response
messages_baseline = [
    SystemMessage(content=f"You are a Redis University course advisor.\n\n{baseline_context}"),
    HumanMessage(content=test_query),
]
response_baseline = llm.invoke(messages_baseline)

# Optimized response
messages_optimized = [
    SystemMessage(
        content=f"You are a Redis University course advisor.\n\n{optimized_context}"
    ),
    HumanMessage(content=test_query),
]
response_optimized = llm.invoke(messages_optimized)

print(
    f"""BASELINE RESPONSE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_baseline.content}

OPTIMIZED RESPONSE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
{response_optimized.content}
"""
)

08:43:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


08:43:37 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


BASELINE RESPONSE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
In the Redis University course catalog, there isn't a specific machine learning course labeled as "beginner." However, if you're interested in machine learning and are looking for a foundational course, you might consider starting with the Computer Science Department's RU101: Introduction to Redis Data Structures. This course is beginner-friendly and can provide a good foundation in understanding data structures, which is essential for working with machine learning algorithms.

For a more direct introduction to machine learning concepts, you might want to look outside the Redis University catalog or consider CS007: Machine Learning, which is labeled as advanced but covers introductory topics in machine learning algorithms and applications.

OPTIMIZED RESPONSE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
At Redis University, we currently do not have any beginn

### Step 5: Iterate - Refine Based on Results

Based on the measurements:

**Quality Assessment:**
- ✅ **Relevance:** Both approaches retrieve relevant ML courses
- ✅ **Completeness:** Both mention prerequisites and difficulty levels
- ✅ **Efficiency:** Optimized version uses fewer tokens ({optimized_tokens} vs {baseline_tokens})
- ✅ **Accuracy:** Both provide correct course information

**Decision:** The optimized hybrid approach meets all quality targets while reducing token usage.

**Next Iteration:** Test with more queries to ensure consistency across different query types.

---

### Key Takeaways: Quality Optimization

1. **Define Domain-Specific Metrics** - Don't rely on generic benchmarks
2. **Measure Systematically** - Baseline → Experiment → Measure → Iterate
3. **Balance Trade-offs** - Relevance vs. Efficiency, Completeness vs. Token Budget
4. **Test Before Deployment** - Validate with real queries from your domain
5. **Iterate Continuously** - Quality optimization is ongoing, not one-time

**The Engineering Mindset:**
- Context quality is measurable
- Optimization is systematic, not guesswork
- Domain-specific metrics matter more than generic benchmarks
- Testing and iteration are essential

---

## 📝 Summary

You've mastered practical context engineering:

**Part 1: The Engineering Mindset**
- ✅ Context is data requiring engineering discipline
- ✅ Naive approaches fail in real-world applications
- ✅ Engineering mindset: Requirements → Transformation → Quality → Testing

**Part 2: Data Engineering Pipeline**
- ✅ Extract → Clean → Transform → Optimize → Store
- ✅ Concrete examples with course data
- ✅ Token optimization techniques

**Part 3: Engineering Approaches**
- ✅ RAG (Semantic Search)
- ✅ Structured Views (Pre-Computed Summaries)
- ✅ Hybrid (Best of Both Worlds)
- ✅ Decision framework for choosing approaches

**Part 4: Chunking Strategies**
- ✅ When to chunk (critical first question)
- ✅ Four strategies with LangChain integration
- ✅ Trade-offs and decision criteria

**Part 5: Context Pipeline Architectures**
- ✅ Request-Time, Batch, Event-Driven
- ✅ Batch processing example with data
- ✅ Decision framework for architecture selection

**Part 6: Quality Optimization**
- ✅ Domain-specific quality metrics
- ✅ Systematic optimization process
- ✅ Baseline → Experiment → Measure → Iterate

**You're now ready to engineer practical context for any domain!** 🎉

---

## 🚀 What's Next?

### Section 3: Memory Systems for Context Engineering

Now that you can engineer high-quality retrieved context, you'll learn to manage conversation context:
- **Working Memory:** Track conversation history within a session
- **Long-term Memory:** Remember user preferences across sessions
- **LangGraph Integration:** Manage stateful workflows with checkpointing
- **Redis Agent Memory Server:** Automatic memory extraction and retrieval

### Section 4: Tool Use and Agents

After adding memory, you'll build complete autonomous agents:
- **Tool Calling:** Let the AI use functions (search, enroll, check prerequisites)
- **LangGraph State Management:** Orchestrate complex multi-step workflows
- **Agent Reasoning:** Plan and execute multi-step tasks
- **Practical Patterns:** Error handling, retries, and monitoring

```
Section 1: Context Engineering Fundamentals
    ↓
Section 2, NB1: RAG Fundamentals
    ↓
Section 2, NB2: Crafting and Optimizing Context ← You are here
    ↓
Section 3: Memory Systems for Context Engineering ← Next
    ↓
Section 4: Tool Use and Agents (Complete System)
```

---

## Additional Resources

**Chunking Strategies:**
- [LangChain Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)
- [LlamaIndex Node Parsers](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/)

**Data Engineering for LLMs:**
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/prompt-engineering)
- [Anthropic Prompt Engineering](https://docs.anthropic.com/claude/docs/prompt-engineering)

**Vector Databases:**
- [Redis Vector Search Documentation](https://redis.io/docs/stack/search/reference/vectors/)
- [RedisVL Python Library](https://github.com/RedisVentures/redisvl)

